# modifying for fruits

In [ ]:
import os
import json
import pandas as pd
import numpy as np
import base64
import requests
import asyncio
import aiohttp
import nest_asyncio
import time
from PIL import Image
nest_asyncio.apply()
global all_data_results
import random
import os
import pandas as pd
from sklearn.utils import shuffle
from fuzzywuzzy import process
import io
# Set the base directory
base_directory = 'data/fruits-images-dataset-object-detection/Train File/Train File'
# FOR API
max_requests_per_half_minute = 20

num_instances_per_class=10

# Define the list of expected class labels
expected_classes = ['Banana', 'Jackfruit', 'Mango', 'Litchi', 'Hog Plum', 'Papaya', 'Grapes', 'Apple', 'Orange', 'Guava']

# Create empty lists to store file paths and labels
file_paths = []
labels = []

# Loop through all files in the directory
for filename in os.listdir(base_directory):
    if filename.endswith('.jpg') or filename.endswith('.jpeg'):
        # Extract the label from the filename
        label = filename.split('_')[1].split('.')[0]
        
        # Find the closest match among expected classes using fuzzy matching
        closest_match = process.extractOne(label, expected_classes)
        
        # Get the standardized label (closest match)
        standardized_label = closest_match[0]
        
        # Append the file path and standardized label to the respective lists
        file_paths.append(os.path.join(base_directory, filename))
        labels.append(standardized_label)

# Create a dataframe from the lists
data = pd.DataFrame({0: file_paths, 1: labels})

# Get the unique classes
unique_classes = data[1].unique()

# Create a new dataframe with sampled data
sampled_data = pd.DataFrame(columns=[0, 1])

# Sample uniformly from each class
for cls in unique_classes:
    class_data = data[data[1] == cls].sample(n=num_instances_per_class, random_state=42)
    sampled_data = pd.concat([sampled_data, class_data], ignore_index=True)

# Shuffle the sampled data
all_data = shuffle(sampled_data, random_state=42).reset_index(drop=True)


def load_image(image_path: str) -> str:
    """Load image from file and encode it as base64."""
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')


api_key = os.getenv("OPENAI_API_KEY")
model ="gpt-4o-2024-05-13" #"gpt-4-turbo"
url = "https://api.openai.com/v1/chat/completions"
headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {api_key}"
}

vision_prompt = f"""
Given the image, identify the class. Use the following list of possible classes for your prediction It should be one of the : {expected_classes}. Be attentive to subtle details as some classes may appear similar. Provide your answer in the following JSON format:
{{"prediction": "class_name"}}
Replace "class_name" with the appropriate class from the list above based on your analysis of the image.
"""


request_times = []

async def get_image_informations(session, inputs: dict) -> dict:
    """Invoke model with image and prompt."""
    global request_times
    
    while True:
        current_time = time.time()
        
        # Remove old request times outside the current time window
        while request_times and request_times[0] <= current_time - 30:
            request_times.pop(0)
        
        if len(request_times) < max_requests_per_half_minute:
            request_times.append(current_time)
            break
        else:
            # If the rate limit is exceeded, wait for a short duration before checking again
            await asyncio.sleep(1)
    
    
    payload = {
        "model": model,
        "response_format":{ "type": "json_object" },
        "messages": [
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": vision_prompt},
                    *inputs['examples'],
                    {
                        "type": "image_url",
                        "image_url": {
                            "url": f"data:image/jpeg;base64,{inputs['image']}",
                            "detail": "low"
                        }
                    }
                ]
            }
        ],
        "max_tokens": 4096
    }
    async with session.post(url, headers=headers, json=payload) as response:
        result = await response.json()
        print(result)
        if "choices" in result and result["choices"]:
            return result
        else:
            raise Exception("Unexpected API response format")

async def process_image(session, i, number_of_shots, all_data_results):
    try:
        image_path = all_data[0][i]
        image_base64 = load_image(image_path)
        examples = []

        # Get the number of rows in all_data
        num_rows = len(all_data)

        # Select random indices from the dataframe
        random_indices = random.sample(range(num_rows), number_of_shots)

        for j in random_indices:
            example_image_path = all_data[0][j]
            example_image_base64 = load_image(example_image_path)
            examples.append({"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{example_image_base64}", "detail": "low"}})
            examples.append({"type": "text", "text": f'{{"prediction": "{all_data.at[j, 1]}"}}' })


        result = await get_image_informations(session, {"image": image_base64, "examples": examples})
        prediction = result["choices"][0]['message']['content']
        all_data_results.at[i, f"# of Shots {number_of_shots}"] = json.loads(prediction)['prediction']

    except Exception as e:
        print(f"Error processing {all_data[0][i]}: {str(e)}")
        all_data_results.at[i, f"# of Shots {number_of_shots}"] = 'NA'


async def process_images_for_shots(number_of_shots, all_data_results):
    async with aiohttp.ClientSession() as session:
        tasks = []
        for i in range(len(all_data)):#range(0, min(samples_to_run, len(all_data))):
            task = asyncio.ensure_future(process_image(session, i, number_of_shots, all_data_results))
            tasks.append(task)
        await asyncio.gather(*tasks)

async def main():
    all_data_results = all_data.copy(deep=True)
    all_data_results.columns = all_data_results.columns.map(str)

    for number_of_shots in [0]:#, 1, 2, 4, 8]:
        await process_images_for_shots(number_of_shots, all_data_results)
    all_data_results.to_csv("01-Fruit.csv")
    # print(all_data_results.iloc[9:16])
    print("done")

asyncio.run(main())

In [ ]:
all_data_results= pd.read_csv("01-Fruit-high-setting.csv")
all_data_results.head()
#calculate wrong predictions
wrong_predictions = all_data_results[all_data_results['1'] != all_data_results['# of Shots 0']]
print(f"Number of wrong predictions: {len(wrong_predictions)}")
#now printing the samples which are predicted wrong
wrong_predictions.head()

In [ ]:
import os;os.getenv("OPENAI_API_KEY")

# 2. drowsy


In [ ]:
import os
import json
import pandas as pd
import numpy as np
import base64
import requests
import asyncio
import aiohttp
import nest_asyncio
import time

nest_asyncio.apply()
global all_data_results
import random
import os
import pandas as pd
from sklearn.utils import shuffle
from fuzzywuzzy import process

# Set the base directory
base_directory = 'data/Drowsy_datset/train'

# FOR API
max_requests_per_half_minute = 20
num_instances_per_class = 50

# Define the list of expected class labels
expected_classes = ['DROWSY', 'NATURAL']

# Create empty lists to store file paths and labels
file_paths = []
labels = []

# Loop through all subdirectories in the base directory
for subdir in os.listdir(base_directory):
    if subdir in expected_classes:
        subdir_path = os.path.join(base_directory, subdir)
        # Loop through all files in the subdirectory
        for filename in os.listdir(subdir_path):
            if filename.endswith('.jpg') or filename.endswith('.png'):
                # Append the file path and label to the respective lists
                file_paths.append(os.path.join(subdir_path, filename))
                labels.append(subdir)

# Create a dataframe from the lists
data = pd.DataFrame({0: file_paths, 1: labels})

# Get the unique classes
unique_classes = data[1].unique()

# Create a new dataframe with sampled data
sampled_data = pd.DataFrame(columns=[0, 1])

# Sample uniformly from each class
for cls in unique_classes:
    class_data = data[data[1] == cls].sample(n=num_instances_per_class, random_state=42)
    sampled_data = pd.concat([sampled_data, class_data], ignore_index=True)

# Shuffle the sampled data
all_data = shuffle(sampled_data, random_state=42).reset_index(drop=True)



def load_image(image_path: str) -> str:
    """Load image from file and encode it as base64."""
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

api_key = os.getenv("OPENAI_API_KEY")
model ="gpt-4o-2024-05-13" #"gpt-4-turbo"
url = "https://api.openai.com/v1/chat/completions"
headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {api_key}"
}

vision_prompt = f"""
Given the image, identify the class. Use the following list of possible classes for your prediction It should be one of the : {expected_classes}. Be attentive to subtle details as some classes may appear similar. Provide your answer in the following JSON format:
{{"prediction": "class_name"}}
Replace "class_name" with the appropriate class from the list above based on your analysis of the image.
"""


request_times = []

async def get_image_informations(session, inputs: dict) -> dict:
    """Invoke model with image and prompt."""
    global request_times
    
    while True:
        current_time = time.time()
        
        # Remove old request times outside the current time window
        while request_times and request_times[0] <= current_time - 30:
            request_times.pop(0)
        
        if len(request_times) < max_requests_per_half_minute:
            request_times.append(current_time)
            break
        else:
            # If the rate limit is exceeded, wait for a short duration before checking again
            await asyncio.sleep(1)
    
    
    payload = {
        "model": model,
        "response_format":{ "type": "json_object" },
        "messages": [
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": vision_prompt},
                    *inputs['examples'],
                    {
                        "type": "image_url",
                        "image_url": {
                            "url": f"data:image/jpeg;base64,{inputs['image']}",
                            "detail": "high"
                        }
                    }
                ]
            }
        ],
        "max_tokens": 4096
    }
    async with session.post(url, headers=headers, json=payload) as response:
        result = await response.json()
        print(result)
        if "choices" in result and result["choices"]:
            return result
        else:
            raise Exception("Unexpected API response format")

async def process_image(session, i, number_of_shots, all_data_results):
    try:
        image_path = all_data[0][i]
        image_base64 = load_image(image_path)
        examples = []

        # Get the number of rows in all_data
        num_rows = len(all_data)

        # Select random indices from the dataframe
        random_indices = random.sample(range(num_rows), number_of_shots)

        for j in random_indices:
            example_image_path = all_data[0][j]
            example_image_base64 = load_image(example_image_path)
            examples.append({"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{example_image_base64}", "detail": "high"}})
            examples.append({"type": "text", "text": f'{{"prediction": "{all_data.at[j, 1]}"}}' })


        result = await get_image_informations(session, {"image": image_base64, "examples": examples})
        prediction = result["choices"][0]['message']['content']
        all_data_results.at[i, f"# of Shots {number_of_shots}"] = json.loads(prediction)['prediction']

    except Exception as e:
        print(f"Error processing {all_data[0][i]}: {str(e)}")
        all_data_results.at[i, f"# of Shots {number_of_shots}"] = 'NA'


async def process_images_for_shots(number_of_shots, all_data_results):
    async with aiohttp.ClientSession() as session:
        tasks = []
        for i in range(len(all_data)):#range(0, min(samples_to_run, len(all_data))):
            task = asyncio.ensure_future(process_image(session, i, number_of_shots, all_data_results))
            tasks.append(task)
        await asyncio.gather(*tasks)

async def main():
    all_data_results = all_data.copy(deep=True)
    all_data_results.columns = all_data_results.columns.map(str)

    for number_of_shots in [0]:#, 1, 2, 4, 8]:
        await process_images_for_shots(number_of_shots, all_data_results)
    all_data_results.to_csv("02-Drowsy.csv")
    # print(all_data_results.iloc[9:16])
    print("done")

asyncio.run(main())

# 3. crop disease

In [ ]:
import os
import json
import pandas as pd
import numpy as np
import base64
import requests
import asyncio
import aiohttp
import nest_asyncio
import time

nest_asyncio.apply()
global all_data_results
import random
import os
import pandas as pd
from sklearn.utils import shuffle
from fuzzywuzzy import process

# Set the base directory
base_directory = 'data/crop-disease'

# FOR API
max_requests_per_half_minute = 20

total_samples_to_check=100
# Define the list of expected class labels
expected_classes = ['Clubroot', 'Canker', 'Anthracnose', 'Blossom End Rot',
       'Verticillium', 'Powdery Mildew', 'Leaf Spots', 'Mosaic Virus',
       'Botrytis', 'Fire Blight', 'Cedar Apple Rust', 'Gray Mold',
       'Fusarium', 'Nematodes', 'Apple Scab', 'Crown Gall', 'Black Spot',
       'Downy Mildew', 'Blight', 'Brown Rot']

num_instances_per_class = int(total_samples_to_check/len(expected_classes))
# Create empty lists to store file paths and labels
file_paths = []
labels = []

# Loop through all subdirectories in the base directory
for subdir in os.listdir(base_directory):
    if subdir==".DS_Store":
        continue
    # if subdir in expected_classes:
    subdir_path = os.path.join(base_directory, subdir)
    # Loop through all files in the subdirectory
    for filename in os.listdir(subdir_path):
        if filename.endswith('.jpg') or filename.endswith('.png'):
            # Append the file path and label to the respective lists
            file_paths.append(os.path.join(subdir_path, filename))
            labels.append(subdir)

# Create a dataframe from the lists
data = pd.DataFrame({0: file_paths, 1: labels})

# Get the unique classes
unique_classes = data[1].unique()

# Create a new dataframe with sampled data
sampled_data = pd.DataFrame(columns=[0, 1])

# Sample uniformly from each class
for cls in unique_classes:
    class_data = data[data[1] == cls].sample(n=num_instances_per_class, random_state=42)
    sampled_data = pd.concat([sampled_data, class_data], ignore_index=True)

# Shuffle the sampled data
all_data = shuffle(sampled_data, random_state=42).reset_index(drop=True)



def load_image(image_path: str) -> str:
    """Load image from file and encode it as base64."""
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

api_key = os.getenv("OPENAI_API_KEY")
model ="gpt-4o-2024-05-13" #"gpt-4-turbo"
url = "https://api.openai.com/v1/chat/completions"
headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {api_key}"
}

vision_prompt = f"""
Given the image, identify the class. Use the following list of possible classes for your prediction It should be one of the : {expected_classes}. Be attentive to subtle details as some classes may appear similar. Provide your answer in the following JSON format:
{{"prediction": "class_name"}}
Replace "class_name" with the appropriate class from the list above based on your analysis of the image.
"""


request_times = []

async def get_image_informations(session, inputs: dict) -> dict:
    """Invoke model with image and prompt."""
    global request_times
    
    while True:
        current_time = time.time()
        
        # Remove old request times outside the current time window
        while request_times and request_times[0] <= current_time - 30:
            request_times.pop(0)
        
        if len(request_times) < max_requests_per_half_minute:
            request_times.append(current_time)
            break
        else:
            # If the rate limit is exceeded, wait for a short duration before checking again
            await asyncio.sleep(1)
    
    
    payload = {
        "model": model,
        "response_format":{ "type": "json_object" },
        "messages": [
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": vision_prompt},
                    *inputs['examples'],
                    {
                        "type": "image_url",
                        "image_url": {
                            "url": f"data:image/jpeg;base64,{inputs['image']}",
                            "detail": "high"
                        }
                    }
                ]
            }
        ],
        "max_tokens": 4096
    }
    async with session.post(url, headers=headers, json=payload) as response:
        result = await response.json()
        print(result)
        if "choices" in result and result["choices"]:
            return result
        else:
            raise Exception("Unexpected API response format")

async def process_image(session, i, number_of_shots, all_data_results):
    try:
        image_path = all_data[0][i]
        image_base64 = load_image(image_path)
        examples = []

        # Get the number of rows in all_data
        num_rows = len(all_data)

        # Select random indices from the dataframe
        random_indices = random.sample(range(num_rows), number_of_shots)

        for j in random_indices:
            example_image_path = all_data[0][j]
            example_image_base64 = load_image(example_image_path)
            examples.append({"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{example_image_base64}", "detail": "high"}})
            examples.append({"type": "text", "text": f'{{"prediction": "{all_data.at[j, 1]}"}}' })


        result = await get_image_informations(session, {"image": image_base64, "examples": examples})
        prediction = result["choices"][0]['message']['content']
        all_data_results.at[i, f"# of Shots {number_of_shots}"] = json.loads(prediction)['prediction']

    except Exception as e:
        print(f"Error processing {all_data[0][i]}: {str(e)}")
        all_data_results.at[i, f"# of Shots {number_of_shots}"] = 'NA'


async def process_images_for_shots(number_of_shots, all_data_results):
    async with aiohttp.ClientSession() as session:
        tasks = []
        for i in range(len(all_data)):#range(0, min(samples_to_run, len(all_data))):
            task = asyncio.ensure_future(process_image(session, i, number_of_shots, all_data_results))
            tasks.append(task)
        await asyncio.gather(*tasks)

async def main():
    all_data_results = all_data.copy(deep=True)
    all_data_results.columns = all_data_results.columns.map(str)

    for number_of_shots in [0]:#, 1, 2, 4, 8]:
        await process_images_for_shots(number_of_shots, all_data_results)
    all_data_results.to_csv("03-Crop.csv")
    # print(all_data_results.iloc[9:16])
    print("done")

asyncio.run(main())

# 4. glaucoma

In [ ]:
import os
import json
import pandas as pd
import numpy as np
import base64
import requests
import asyncio
import aiohttp
import nest_asyncio
import time

nest_asyncio.apply()
global all_data_results
import random
import os
import pandas as pd
from sklearn.utils import shuffle
from fuzzywuzzy import process

# Set the base directory
base_directory = 'DIRECTORY/data/glaucoma-dataset/Dataset_New/Dataset_New/ACRIMA'

# FOR API
max_requests_per_half_minute = 20

total_samples_to_check=100
# Define the list of expected class labels
expected_classes = ['glaucoma', 'normal']

num_instances_per_class = int(total_samples_to_check/len(expected_classes))
# Create empty lists to store file paths and labels
file_paths = []
labels = []

# Loop through all subdirectories in the base directory
for subdir in os.listdir(base_directory):
    if subdir==".DS_Store":
        continue
    # if subdir in expected_classes:
    subdir_path = os.path.join(base_directory, subdir)
    # Loop through all files in the subdirectory
    for filename in os.listdir(subdir_path):
        if filename.endswith('.jpg') or filename.endswith('.png'):
            # Append the file path and label to the respective lists
            file_paths.append(os.path.join(subdir_path, filename))
            labels.append(subdir)

# Create a dataframe from the lists
data = pd.DataFrame({0: file_paths, 1: labels})

# Get the unique classes
unique_classes = data[1].unique()

# Create a new dataframe with sampled data
sampled_data = pd.DataFrame(columns=[0, 1])

# Sample uniformly from each class
for cls in unique_classes:
    class_data = data[data[1] == cls].sample(n=num_instances_per_class, random_state=42)
    sampled_data = pd.concat([sampled_data, class_data], ignore_index=True)

# Shuffle the sampled data
all_data = shuffle(sampled_data, random_state=42).reset_index(drop=True)



def load_image(image_path: str) -> str:
    """Load image from file and encode it as base64."""
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

api_key = os.getenv("OPENAI_API_KEY")
model ="gpt-4o-2024-05-13" #"gpt-4-turbo"
url = "https://api.openai.com/v1/chat/completions"
headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {api_key}"
}

vision_prompt = f"""
Given the image, identify the class. Use the following list of possible classes for your prediction It should be one of the : {expected_classes}. Be attentive to subtle details as some classes may appear similar. Provide your answer in the following JSON format:
{{"prediction": "class_name"}}
Replace "class_name" with the appropriate class from the list above based on your analysis of the image.
"""


request_times = []

async def get_image_informations(session, inputs: dict) -> dict:
    """Invoke model with image and prompt."""
    global request_times
    
    while True:
        current_time = time.time()
        
        # Remove old request times outside the current time window
        while request_times and request_times[0] <= current_time - 30:
            request_times.pop(0)
        
        if len(request_times) < max_requests_per_half_minute:
            request_times.append(current_time)
            break
        else:
            # If the rate limit is exceeded, wait for a short duration before checking again
            await asyncio.sleep(1)
    
    
    payload = {
        "model": model,
        "response_format":{ "type": "json_object" },
        "messages": [
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": vision_prompt},
                    *inputs['examples'],
                    {
                        "type": "image_url",
                        "image_url": {
                            "url": f"data:image/jpeg;base64,{inputs['image']}",
                            "detail": "high"
                        }
                    }
                ]
            }
        ],
        "max_tokens": 4096
    }
    async with session.post(url, headers=headers, json=payload) as response:
        result = await response.json()
        print(result)
        if "choices" in result and result["choices"]:
            return result
        else:
            raise Exception("Unexpected API response format")

async def process_image(session, i, number_of_shots, all_data_results):
    try:
        image_path = all_data[0][i]
        image_base64 = load_image(image_path)
        examples = []

        # Get the number of rows in all_data
        num_rows = len(all_data)

        # Select random indices from the dataframe
        random_indices = random.sample(range(num_rows), number_of_shots)

        for j in random_indices:
            example_image_path = all_data[0][j]
            example_image_base64 = load_image(example_image_path)
            examples.append({"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{example_image_base64}", "detail": "high"}})
            examples.append({"type": "text", "text": f'{{"prediction": "{all_data.at[j, 1]}"}}' })


        result = await get_image_informations(session, {"image": image_base64, "examples": examples})
        prediction = result["choices"][0]['message']['content']
        all_data_results.at[i, f"# of Shots {number_of_shots}"] = json.loads(prediction)['prediction']

    except Exception as e:
        print(f"Error processing {all_data[0][i]}: {str(e)}")
        all_data_results.at[i, f"# of Shots {number_of_shots}"] = 'NA'


async def process_images_for_shots(number_of_shots, all_data_results):
    async with aiohttp.ClientSession() as session:
        tasks = []
        for i in range(len(all_data)):#range(0, min(samples_to_run, len(all_data))):
            task = asyncio.ensure_future(process_image(session, i, number_of_shots, all_data_results))
            tasks.append(task)
        await asyncio.gather(*tasks)

async def main():
    all_data_results = all_data.copy(deep=True)
    all_data_results.columns = all_data_results.columns.map(str)

    for number_of_shots in [0,1, 2, 4]:#, 1, 2, 4, 8]:
        await process_images_for_shots(number_of_shots, all_data_results)
    all_data_results.to_csv("04-Glaucoma.csv")
    # print(all_data_results.iloc[9:16])
    print("done")

asyncio.run(main())

# 5. CT

In [ ]:
import os
import json
import pandas as pd
import numpy as np
import base64
import requests
import asyncio
import aiohttp
import nest_asyncio
import time

nest_asyncio.apply()
global all_data_results
import random
import os
import pandas as pd
from sklearn.utils import shuffle
from fuzzywuzzy import process

# Set the base directory
base_directory = 'DIRECTORY/data/computed-tomography-ct-of-the-brain/files'

# FOR API
max_requests_per_half_minute = 20

total_samples_to_check=100
# Define the list of expected class labels
expected_classes = ['aneurysm', 'cancer', 'tumor']

num_instances_per_class = int(total_samples_to_check/len(expected_classes))
# Create empty lists to store file paths and labels
file_paths = []
labels = []

# Loop through all subdirectories in the base directory
for subdir in os.listdir(base_directory):
    if subdir==".DS_Store":
        continue
    # if subdir in expected_classes:
    subdir_path = os.path.join(base_directory, subdir)
    # Loop through all files in the subdirectory
    for filename in os.listdir(subdir_path):
        if filename.endswith('.jpg') or filename.endswith('.png'):
            # Append the file path and label to the respective lists
            file_paths.append(os.path.join(subdir_path, filename))
            labels.append(subdir)

# Create a dataframe from the lists
data = pd.DataFrame({0: file_paths, 1: labels})

# Get the unique classes
unique_classes = data[1].unique()

# Create a new dataframe with sampled data
sampled_data = pd.DataFrame(columns=[0, 1])

# Sample uniformly from each class
for cls in unique_classes:
    class_data = data[data[1] == cls].sample(n=num_instances_per_class, random_state=42)
    sampled_data = pd.concat([sampled_data, class_data], ignore_index=True)

# Shuffle the sampled data
all_data = shuffle(sampled_data, random_state=42).reset_index(drop=True)



def load_image(image_path: str) -> str:
    """Load image from file and encode it as base64."""
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

api_key = os.getenv("OPENAI_API_KEY")
model ="gpt-4o-2024-05-13" #"gpt-4-turbo"
url = "https://api.openai.com/v1/chat/completions"
headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {api_key}"
}

vision_prompt = f"""
Given the image, identify the class. Use the following list of possible classes for your prediction It should be one of the : {expected_classes}. Be attentive to subtle details as some classes may appear similar. Provide your answer in the following JSON format:
{{"prediction": "class_name"}}
Replace "class_name" with the appropriate class from the list above based on your analysis of the image.
"""


request_times = []

async def get_image_informations(session, inputs: dict) -> dict:
    """Invoke model with image and prompt."""
    global request_times
    
    while True:
        current_time = time.time()
        
        # Remove old request times outside the current time window
        while request_times and request_times[0] <= current_time - 30:
            request_times.pop(0)
        
        if len(request_times) < max_requests_per_half_minute:
            request_times.append(current_time)
            break
        else:
            # If the rate limit is exceeded, wait for a short duration before checking again
            await asyncio.sleep(1)
    
    
    payload = {
        "model": model,
        "response_format":{ "type": "json_object" },
        "messages": [
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": vision_prompt},
                    *inputs['examples'],
                    {
                        "type": "image_url",
                        "image_url": {
                            "url": f"data:image/jpeg;base64,{inputs['image']}",
                            "detail": "high"
                        }
                    }
                ]
            }
        ],
        "max_tokens": 4096
    }
    async with session.post(url, headers=headers, json=payload) as response:
        result = await response.json()
        print(result)
        if "choices" in result and result["choices"]:
            return result
        else:
            raise Exception("Unexpected API response format")

async def process_image(session, i, number_of_shots, all_data_results):
    try:
        image_path = all_data[0][i]
        image_base64 = load_image(image_path)
        examples = []

        # Get the number of rows in all_data
        num_rows = len(all_data)

        # Select random indices from the dataframe
        random_indices = random.sample(range(num_rows), number_of_shots)

        for j in random_indices:
            example_image_path = all_data[0][j]
            example_image_base64 = load_image(example_image_path)
            examples.append({"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{example_image_base64}", "detail": "high"}})
            examples.append({"type": "text", "text": f'{{"prediction": "{all_data.at[j, 1]}"}}' })


        result = await get_image_informations(session, {"image": image_base64, "examples": examples})
        prediction = result["choices"][0]['message']['content']
        all_data_results.at[i, f"# of Shots {number_of_shots}"] = json.loads(prediction)['prediction']

    except Exception as e:
        print(f"Error processing {all_data[0][i]}: {str(e)}")
        all_data_results.at[i, f"# of Shots {number_of_shots}"] = 'NA'


async def process_images_for_shots(number_of_shots, all_data_results):
    async with aiohttp.ClientSession() as session:
        tasks = []
        for i in range(len(all_data)):#range(0, min(samples_to_run, len(all_data))):
            task = asyncio.ensure_future(process_image(session, i, number_of_shots, all_data_results))
            tasks.append(task)
        await asyncio.gather(*tasks)

async def main():
    all_data_results = all_data.copy(deep=True)
    all_data_results.columns = all_data_results.columns.map(str)

    for number_of_shots in [0]:#, 1, 2, 4, 8]:
        await process_images_for_shots(number_of_shots, all_data_results)
    all_data_results.to_csv("05-CT.csv")
    # print(all_data_results.iloc[9:16])
    print("done")

asyncio.run(main())

# 6. Captions

In [ ]:
import os
import json
import pandas as pd
import numpy as np
import base64
import requests
import asyncio
import aiohttp
import nest_asyncio
import time

nest_asyncio.apply()
global all_data_results
import random
import os
import pandas as pd
from sklearn.utils import shuffle
from fuzzywuzzy import process

import os
import pandas as pd
from sklearn.utils import shuffle

# Set the base directory for images
base_directory_images = 'DIRECTORY/data/flickr-image-caption/images'

# Set the path for the captions file
captions_file = 'DIRECTORY/data/flickr-image-caption/captions.txt'

# FOR API
max_requests_per_half_minute = 20
total_samples_to_check = 100

# Read the captions file
captions_df = pd.read_csv(captions_file)

# Sample the required number of rows
sampled_captions = captions_df.sample(n=total_samples_to_check, random_state=42)

# Create empty lists to store file paths and captions
file_paths = []
captions = []

# Iterate over the sampled rows
for index, row in sampled_captions.iterrows():
    image_file = row['image']
    image_caption = row['caption']
    
    # Construct the full file path
    file_path = os.path.join(base_directory_images, image_file)
    
    # Check if the file exists
    if os.path.isfile(file_path):
        file_paths.append(file_path)
        captions.append(image_caption)

# Create a dataframe from the lists
data = pd.DataFrame({0: file_paths, 1: captions})

# Shuffle the data
all_data = shuffle(data, random_state=42).reset_index(drop=True)


def load_image(image_path: str) -> str:
    """Load image from file and encode it as base64."""
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

api_key = os.getenv("OPENAI_API_KEY")
model ="gpt-4o-2024-05-13" #"gpt-4-turbo"
url = "https://api.openai.com/v1/chat/completions"
headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {api_key}"
}

# vision_prompt = f"""
# Given the image, identify the class of the fruit presented. Use the following list of possible classes for your prediction It should be one of the : {expected_classes}. Be attentive to subtle details as some classes may appear similar. Provide your answer in the following JSON format:
# {{"prediction": "class_name"}}
# Replace "class_name" with the appropriate class from the list above based on your analysis of the image.
# """
vision_prompt = f"""
Given the image, write a short caption for it in a very short single line. Provide your answer in the following JSON format:
{{"prediction": "caption ."}}
Replace "caption" with the appropriate caption based on your analysis of the image. and see i put a full stop (.) at the end of the caption after a space. 
"""

request_times = []

async def get_image_informations(session, inputs: dict) -> dict:
    """Invoke model with image and prompt."""
    global request_times
    
    while True:
        current_time = time.time()
        
        # Remove old request times outside the current time window
        while request_times and request_times[0] <= current_time - 30:
            request_times.pop(0)
        
        if len(request_times) < max_requests_per_half_minute:
            request_times.append(current_time)
            break
        else:
            # If the rate limit is exceeded, wait for a short duration before checking again
            await asyncio.sleep(1)
    
    
    payload = {
        "model": model,
        "response_format":{ "type": "json_object" },
        "messages": [
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": vision_prompt},
                    *inputs['examples'],
                    {
                        "type": "image_url",
                        "image_url": {
                            "url": f"data:image/jpeg;base64,{inputs['image']}",
                            "detail": "high"
                        }
                    }
                ]
            }
        ],
        "max_tokens": 4096
    }
    async with session.post(url, headers=headers, json=payload) as response:
        result = await response.json()
        print(result)
        if "choices" in result and result["choices"]:
            return result
        else:
            raise Exception("Unexpected API response format")

async def process_image(session, i, number_of_shots, all_data_results):
    try:
        image_path = all_data[0][i]
        image_base64 = load_image(image_path)
        examples = []

        # Get the number of rows in all_data
        num_rows = len(all_data)

        # Select random indices from the dataframe
        random_indices = random.sample(range(num_rows), number_of_shots)

        for j in random_indices:
            example_image_path = all_data[0][j]
            example_image_base64 = load_image(example_image_path)
            examples.append({"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{example_image_base64}", "detail": "high"}})
            examples.append({"type": "text", "text": f'{{"prediction": "{all_data.at[j, 1]}"}}' })


        result = await get_image_informations(session, {"image": image_base64, "examples": examples})
        prediction = result["choices"][0]['message']['content']
        all_data_results.at[i, f"# of Shots {number_of_shots}"] = json.loads(prediction)['prediction']

    except Exception as e:
        print(f"Error processing {all_data[0][i]}: {str(e)}")
        all_data_results.at[i, f"# of Shots {number_of_shots}"] = 'NA'


async def process_images_for_shots(number_of_shots, all_data_results):
    async with aiohttp.ClientSession() as session:
        tasks = []
        for i in range(len(all_data)):#range(0, min(samples_to_run, len(all_data))):
            task = asyncio.ensure_future(process_image(session, i, number_of_shots, all_data_results))
            tasks.append(task)
        await asyncio.gather(*tasks)

async def main():
    all_data_results = all_data.copy(deep=True)
    all_data_results.columns = all_data_results.columns.map(str)

    for number_of_shots in [0]:#, 1, 2, 4]:#, 1, 2, 4, 8]:
        await process_images_for_shots(number_of_shots, all_data_results)
    all_data_results.to_csv("06-Captions.csv")
    # print(all_data_results.iloc[9:16])
    print("done")

asyncio.run(main())

# 7. vqa


In [ ]:
import os
import json
import pandas as pd
import numpy as np
import base64
import requests
import asyncio
import aiohttp
import nest_asyncio
import time
import random

nest_asyncio.apply()
global all_data_results

from sklearn.utils import shuffle

# Set the base directory for images
base_directory = 'DIRECTORY/data/VQA'
images_directory = os.path.join(base_directory, 'images')

# Set the path for the data file
data_file = os.path.join(base_directory, 'data.csv')

# Read the list of possible answers from the answer_space.txt file
with open(os.path.join(base_directory, 'answer_space.txt'), 'r') as f:
    answer_space = f.read().splitlines()

# FOR API
max_requests_per_half_minute = 20
total_samples_to_check = 100

# Read the data file
data_df = pd.read_csv(data_file)

# Sample the required number of rows
sampled_data = data_df.sample(n=total_samples_to_check, random_state=42)
# the answers are comma seperated
sampled_data['answer'] = sampled_data['answer'].apply(lambda x: x.split(',')[0])

# Create a dataframe from the sampled data
data = pd.DataFrame({'file_path': [os.path.join(images_directory, f"{row['image_id']}.png") for _, row in sampled_data.iterrows()],
                     'question': sampled_data['question'],
                     'answer': sampled_data['answer']})

# Shuffle the data
all_data = shuffle(data, random_state=42).reset_index(drop=True)

def load_image(image_path: str) -> str:
    """Load image from file and encode it as base64."""
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

api_key = os.getenv("OPENAI_API_KEY")
model ="gpt-4o-2024-05-13" #"gpt-4-turbo"
url = "https://api.openai.com/v1/chat/completions"
headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {api_key}"
}

vision_prompt = f"""
Given the image and question, provide the appropriate answer from the following list of possible answers:
{answer_space}

Provide your answer in the following JSON format:
prediction: "<answer>"
Replace "<answer>" with the appropriate answer from the list above based on your analysis of the image and question. The question is: {{question}}
"""

request_times = []

async def get_image_informations(session, inputs: dict) -> dict:
    """Invoke model with image and prompt."""
    global request_times
    
    while True:
        current_time = time.time()
        
        # Remove old request times outside the current time window
        while request_times and request_times[0] <= current_time - 30:
            request_times.pop(0)
        
        if len(request_times) < max_requests_per_half_minute:
            request_times.append(current_time)
            break
        else:
            # If the rate limit is exceeded, wait for a short duration before checking again
            await asyncio.sleep(1)
    
    
    payload = {
        "model": model,
        "response_format":{ "type": "json_object" },
        "messages": [
            {
                "role": "user",
                "content": [
                    *inputs['examples'],
                    {"type": "text", "text": vision_prompt.format(question=inputs['question'])},

                    {
                        "type": "image_url",
                        "image_url": {
                            "url": f"data:image/jpeg;base64,{inputs['image']}",
                            "detail": "high"
                        }
                    }
                ]
            }
        ],
        "max_tokens": 4096
    }
    async with session.post(url, headers=headers, json=payload) as response:
        result = await response.json()
        print(result)
        if "choices" in result and result["choices"]:
            return result
        else:
            raise Exception("Unexpected API response format")

async def process_image(session, i, number_of_shots, all_data_results):
    try:
        image_path = all_data['file_path'][i]
        question = all_data['question'][i]
        image_base64 = load_image(image_path)
        examples = []

        # Get the number of rows in all_data
        num_rows = len(all_data)

        # Select random indices from the dataframe
        # random_indices = random.sample(range(num_rows), number_of_shots)
        random_indices = random.sample([idx for idx in range(num_rows) if idx != i], number_of_shots)

        for j in random_indices:
            example_image_path = all_data['file_path'][j]
            example_image_base64 = load_image(example_image_path)
            example_question = all_data['question'][j]
            examples.append({"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{example_image_base64}", "detail": "high"}})
            examples.append({"type": "text", "text": f"Question: {example_question}"})
            examples.append({"type": "text", "text": f'{{"prediction": "{all_data.at[j, "answer"]}"}}' })


        result = await get_image_informations(session, {"image": image_base64, "question": question, "examples": examples})
        prediction = result["choices"][0]['message']['content']
        prediction_dict = json.loads(prediction)
        all_data_results.at[i, f"# of Shots {number_of_shots}"] = prediction_dict['prediction']

    except Exception as e:
        print(f"Error processing {all_data['file_path'][i]}: {str(e)}")
        all_data_results.at[i, f"# of Shots {number_of_shots}"] = 'NA'


async def process_images_for_shots(number_of_shots, all_data_results):
    async with aiohttp.ClientSession() as session:
        tasks = []
        for i in range(len(all_data)):#range(0, min(samples_to_run, len(all_data))):
            task = asyncio.ensure_future(process_image(session, i, number_of_shots, all_data_results))
            tasks.append(task)
        await asyncio.gather(*tasks)

async def main():
    all_data_results = all_data.copy(deep=True)
    all_data_results.columns = all_data_results.columns.map(str)

    for number_of_shots in [0, 1, 2, 4, 8]:#, 1, 2, 4]:#, 1, 2, 4, 8]:
        await process_images_for_shots(number_of_shots, all_data_results)
    all_data_results.to_csv("07-VQA.csv")
    # print(all_data_results.iloc[9:16])
    print("done")

asyncio.run(main())

In [ ]:
import pandas as pd
import numpy as np
import os
import json

# Set the base directory for images
base_directory = 'DIRECTORY/data/VQA'

# Set the path for the data file
data_file = os.path.join(base_directory, 'data.csv')
data_df = pd.read_csv(data_file)

file_read = pd.read_csv("07-VQA-all.csv")

# Merge the data_df with file_read based on the index
merged_df = file_read.merge(data_df[['question', 'answer']], left_index=True, right_index=True)

# Rename the 'answer' column to 'ground_truth'
merged_df = merged_df.rename(columns={'answer': 'ground_truth'})

# Save the updated DataFrame to a new CSV file
merged_df.to_csv("07-VQA-all-with-ground-truth.csv", index=False)

# 8. Soybean



In [ ]:
import os
import json
import pandas as pd
import numpy as np
import base64
import requests
import asyncio
import aiohttp
import nest_asyncio
import time

nest_asyncio.apply()
global all_data_results
import random
import os
import pandas as pd
from sklearn.utils import shuffle
from fuzzywuzzy import process

# Set the base directory
base_directory = 'data/Soybean Seeds'

# FOR API
max_requests_per_half_minute = 100

total_samples_to_check=100
# Define the list of expected class labels
expected_classes = ['Broken', 'Immature soybeans','Intact','Skin-damaged' ,'Spotted' ]

num_instances_per_class = int(total_samples_to_check/len(expected_classes))
# Create empty lists to store file paths and labels
file_paths = []
labels = []

# Loop through all subdirectories in the base directory
for subdir in os.listdir(base_directory):
    if subdir==".DS_Store":
        continue
    # if subdir in expected_classes:
    subdir_path = os.path.join(base_directory, subdir)
    # Loop through all files in the subdirectory
    for filename in os.listdir(subdir_path):
        if filename.endswith('.jpg') or filename.endswith('.png'):
            # Append the file path and label to the respective lists
            file_paths.append(os.path.join(subdir_path, filename))
            labels.append(subdir)

# Create a dataframe from the lists
data = pd.DataFrame({0: file_paths, 1: labels})

# Get the unique classes
unique_classes = data[1].unique()

# Create a new dataframe with sampled data
sampled_data = pd.DataFrame(columns=[0, 1])

# Sample uniformly from each class
for cls in unique_classes:
    class_data = data[data[1] == cls].sample(n=num_instances_per_class, random_state=42)
    sampled_data = pd.concat([sampled_data, class_data], ignore_index=True)

# Shuffle the sampled data
all_data = shuffle(sampled_data, random_state=42).reset_index(drop=True)



def load_image(image_path: str) -> str:
    """Load image from file and encode it as base64."""
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

api_key = os.getenv("OPENAI_API_KEY")
model ="gpt-4o-2024-05-13" #"gpt-4-turbo"
url = "https://api.openai.com/v1/chat/completions"
headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {api_key}"
}

vision_prompt = f"""
Given the image, identify the class. Use the following list of possible classes for your prediction It should be one of the : {expected_classes}. Be attentive to subtle details as some classes may appear similar. Provide your answer in the following JSON format:
{{"prediction": "class_name"}}
Replace "class_name" with the appropriate class from the list above based on your analysis of the image.
"""


request_times = []

async def get_image_informations(session, inputs: dict) -> dict:
    """Invoke model with image and prompt."""
    global request_times
    
    while True:
        current_time = time.time()
        
        # Remove old request times outside the current time window
        while request_times and request_times[0] <= current_time - 30:
            request_times.pop(0)
        
        if len(request_times) < max_requests_per_half_minute:
            request_times.append(current_time)
            break
        else:
            # If the rate limit is exceeded, wait for a short duration before checking again
            await asyncio.sleep(1)
    
    
    payload = {
        "model": model,
        "response_format":{ "type": "json_object" },
        "messages": [
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": vision_prompt},
                    *inputs['examples'],
                    {
                        "type": "image_url",
                        "image_url": {
                            "url": f"data:image/jpeg;base64,{inputs['image']}",
                            "detail": "high"
                        }
                    }
                ]
            }
        ],
        "max_tokens": 4096
    }
    async with session.post(url, headers=headers, json=payload) as response:
        result = await response.json()
        print(result)
        if "choices" in result and result["choices"]:
            return result
        else:
            raise Exception("Unexpected API response format")

async def process_image(session, i, number_of_shots, all_data_results):
    try:
        image_path = all_data[0][i]
        image_base64 = load_image(image_path)
        examples = []

        # Get the number of rows in all_data
        num_rows = len(all_data)

        # Select random indices from the dataframe
        random_indices = random.sample([idx for idx in range(num_rows) if idx != i], number_of_shots)

        for j in random_indices:
            example_image_path = all_data[0][j]
            example_image_base64 = load_image(example_image_path)
            examples.append({"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{example_image_base64}", "detail": "high"}})
            examples.append({"type": "text", "text": f'{{"prediction": "{all_data.at[j, 1]}"}}' })


        result = await get_image_informations(session, {"image": image_base64, "examples": examples})
        prediction = result["choices"][0]['message']['content']
        all_data_results.at[i, f"# of Shots {number_of_shots}"] = json.loads(prediction)['prediction']

    except Exception as e:
        print(f"Error processing {all_data[0][i]}: {str(e)}")
        all_data_results.at[i, f"# of Shots {number_of_shots}"] = 'NA'


async def process_images_for_shots(number_of_shots, all_data_results):
    async with aiohttp.ClientSession() as session:
        tasks = []
        for i in range(len(all_data)):#range(0, min(samples_to_run, len(all_data))):
            task = asyncio.ensure_future(process_image(session, i, number_of_shots, all_data_results))
            tasks.append(task)
        await asyncio.gather(*tasks)

async def main():
    all_data_results = all_data.copy(deep=True)
    all_data_results.columns = all_data_results.columns.map(str)

    for number_of_shots in [0, 1, 2, 4, 8]:
        await process_images_for_shots(number_of_shots, all_data_results)
    all_data_results.to_csv(base_directory.split("/")[-1] + ".csv")
    # print(all_data_results.iloc[9:16])
    print("done")

asyncio.run(main())

# 9. Mangoe Disease


In [ ]:
import os
import json
import pandas as pd
import numpy as np
import base64
import requests
import asyncio
import aiohttp
import nest_asyncio
import time

nest_asyncio.apply()
global all_data_results
import random
import os
import pandas as pd
from sklearn.utils import shuffle
from fuzzywuzzy import process

# Set the base directory
base_directory = 'data/mango-leaf-disease-dataset'

# FOR API
max_requests_per_half_minute = 100

total_samples_to_check=100
# Define the list of expected class labels
expected_classes = ['Anthracnose', 'Bacterial Canker', 'Cutting Weevil', 'Die Back', 'Gall Midge', 'Healthy', 'Powdery Mildew', 'Sooty Mould' ]

num_instances_per_class = int(total_samples_to_check/len(expected_classes))
# Create empty lists to store file paths and labels
file_paths = []
labels = []

# Loop through all subdirectories in the base directory
for subdir in os.listdir(base_directory):
    if subdir==".DS_Store":
        continue
    # if subdir in expected_classes:
    subdir_path = os.path.join(base_directory, subdir)
    # Loop through all files in the subdirectory
    for filename in os.listdir(subdir_path):
        if filename.endswith('.jpg') or filename.endswith('.png'):
            # Append the file path and label to the respective lists
            file_paths.append(os.path.join(subdir_path, filename))
            labels.append(subdir)

# Create a dataframe from the lists
data = pd.DataFrame({0: file_paths, 1: labels})

# Get the unique classes
unique_classes = data[1].unique()

# Create a new dataframe with sampled data
sampled_data = pd.DataFrame(columns=[0, 1])

# Sample uniformly from each class
for cls in unique_classes:
    class_data = data[data[1] == cls].sample(n=num_instances_per_class, random_state=42)
    sampled_data = pd.concat([sampled_data, class_data], ignore_index=True)

# Shuffle the sampled data
all_data = shuffle(sampled_data, random_state=42).reset_index(drop=True)



def load_image(image_path: str) -> str:
    """Load image from file and encode it as base64."""
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

api_key = os.getenv("OPENAI_API_KEY")
model ="gpt-4o-2024-05-13" #"gpt-4-turbo"
url = "https://api.openai.com/v1/chat/completions"
headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {api_key}"
}

vision_prompt = f"""
Given the image, identify the class. Use the following list of possible classes for your prediction It should be one of the : {expected_classes}. Be attentive to subtle details as some classes may appear similar. Provide your answer in the following JSON format:
{{"prediction": "class_name"}}
Replace "class_name" with the appropriate class from the list above based on your analysis of the image.
"""


request_times = []

async def get_image_informations(session, inputs: dict) -> dict:
    """Invoke model with image and prompt."""
    global request_times
    
    while True:
        current_time = time.time()
        
        # Remove old request times outside the current time window
        while request_times and request_times[0] <= current_time - 30:
            request_times.pop(0)
        
        if len(request_times) < max_requests_per_half_minute:
            request_times.append(current_time)
            break
        else:
            # If the rate limit is exceeded, wait for a short duration before checking again
            await asyncio.sleep(1)
    
    
    payload = {
        "model": model,
        "response_format":{ "type": "json_object" },
        "messages": [
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": vision_prompt},
                    *inputs['examples'],
                    {
                        "type": "image_url",
                        "image_url": {
                            "url": f"data:image/jpeg;base64,{inputs['image']}",
                            "detail": "high"
                        }
                    }
                ]
            }
        ],
        "max_tokens": 4096
    }
    async with session.post(url, headers=headers, json=payload) as response:
        result = await response.json()
        print(result)
        if "choices" in result and result["choices"]:
            return result
        else:
            raise Exception("Unexpected API response format")

async def process_image(session, i, number_of_shots, all_data_results):
    try:
        image_path = all_data[0][i]
        image_base64 = load_image(image_path)
        examples = []

        # Get the number of rows in all_data
        num_rows = len(all_data)

        # Select random indices from the dataframe
        random_indices = random.sample([idx for idx in range(num_rows) if idx != i], number_of_shots)

        for j in random_indices:
            example_image_path = all_data[0][j]
            example_image_base64 = load_image(example_image_path)
            examples.append({"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{example_image_base64}", "detail": "high"}})
            examples.append({"type": "text", "text": f'{{"prediction": "{all_data.at[j, 1]}"}}' })


        result = await get_image_informations(session, {"image": image_base64, "examples": examples})
        prediction = result["choices"][0]['message']['content']
        all_data_results.at[i, f"# of Shots {number_of_shots}"] = json.loads(prediction)['prediction']

    except Exception as e:
        print(f"Error processing {all_data[0][i]}: {str(e)}")
        all_data_results.at[i, f"# of Shots {number_of_shots}"] = 'NA'


async def process_images_for_shots(number_of_shots, all_data_results):
    async with aiohttp.ClientSession() as session:
        tasks = []
        for i in range(len(all_data)):#range(0, min(samples_to_run, len(all_data))):
            task = asyncio.ensure_future(process_image(session, i, number_of_shots, all_data_results))
            tasks.append(task)
        await asyncio.gather(*tasks)

async def main():
    all_data_results = all_data.copy(deep=True)
    all_data_results.columns = all_data_results.columns.map(str)

    for number_of_shots in [0, 1, 2, 4, 8]:
        await process_images_for_shots(number_of_shots, all_data_results)
    all_data_results.to_csv(base_directory.split("/")[-1] + ".csv")
    # print(all_data_results.iloc[9:16])
    print("done")

asyncio.run(main())

# 10. Durum Wheat Dataset

In [ ]:
import os
import json
import pandas as pd
import numpy as np
import base64
import requests
import asyncio
import aiohttp
import nest_asyncio
import time

nest_asyncio.apply()
global all_data_results
import random
import os
import pandas as pd
from sklearn.utils import shuffle
from fuzzywuzzy import process

# Set the base directory
base_directory = 'DIRECTORY/AgEval-datasets/Durum_Wheat_Dataset/Dataset2-Durum Wheat Video Images/processed'

# FOR API
max_requests_per_half_minute = 100

total_samples_to_check=100
# Define the list of expected class labels
expected_classes = ['Foreign Matters', 'Starchy Kernels', 'Vitreous Kernels' ]

num_instances_per_class = int(total_samples_to_check/len(expected_classes))
# Create empty lists to store file paths and labels
file_paths = []
labels = []

# Loop through all subdirectories in the base directory
for subdir in os.listdir(base_directory):
    if subdir==".DS_Store":
        continue
    # if subdir in expected_classes:
    subdir_path = os.path.join(base_directory, subdir)
    # Loop through all files in the subdirectory
    for filename in os.listdir(subdir_path):
        if filename.endswith('.jpg') or filename.endswith('.png'):
            # Append the file path and label to the respective lists
            file_paths.append(os.path.join(subdir_path, filename))
            labels.append(subdir)

# Create a dataframe from the lists
data = pd.DataFrame({0: file_paths, 1: labels})

# Get the unique classes
unique_classes = data[1].unique()

# Create a new dataframe with sampled data
sampled_data = pd.DataFrame(columns=[0, 1])

# Sample uniformly from each class
for cls in unique_classes:
    class_data = data[data[1] == cls].sample(n=num_instances_per_class, random_state=42)
    sampled_data = pd.concat([sampled_data, class_data], ignore_index=True)

# Shuffle the sampled data
all_data = shuffle(sampled_data, random_state=42).reset_index(drop=True)



def load_image(image_path: str) -> str:
    """Load image from file and encode it as base64."""
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

api_key = os.getenv("OPENAI_API_KEY")
model ="gpt-4o-2024-05-13" #"gpt-4-turbo"
url = "https://api.openai.com/v1/chat/completions"
headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {api_key}"
}

vision_prompt = f"""
Given the image, identify the class. Use the following list of possible classes for your prediction It should be one of the : {expected_classes}. Be attentive to subtle details as some classes may appear similar. Provide your answer in the following JSON format:
{{"prediction": "class_name"}}
Replace "class_name" with the appropriate class from the list above based on your analysis of the image.
"""


request_times = []

async def get_image_informations(session, inputs: dict) -> dict:
    """Invoke model with image and prompt."""
    global request_times
    
    while True:
        current_time = time.time()
        
        # Remove old request times outside the current time window
        while request_times and request_times[0] <= current_time - 30:
            request_times.pop(0)
        
        if len(request_times) < max_requests_per_half_minute:
            request_times.append(current_time)
            break
        else:
            # If the rate limit is exceeded, wait for a short duration before checking again
            await asyncio.sleep(1)
    
    
    payload = {
        "model": model,
        "response_format":{ "type": "json_object" },
        "messages": [
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": vision_prompt},
                    *inputs['examples'],
                    {
                        "type": "image_url",
                        "image_url": {
                            "url": f"data:image/jpeg;base64,{inputs['image']}",
                            "detail": "high"
                        }
                    }
                ]
            }
        ],
        "max_tokens": 4096
    }
    async with session.post(url, headers=headers, json=payload) as response:
        result = await response.json()
        print(result)
        if "choices" in result and result["choices"]:
            return result
        else:
            raise Exception("Unexpected API response format")

async def process_image(session, i, number_of_shots, all_data_results):
    try:
        image_path = all_data[0][i]
        image_base64 = load_image(image_path)
        examples = []

        # Get the number of rows in all_data
        num_rows = len(all_data)

        # Select random indices from the dataframe
        random_indices = random.sample([idx for idx in range(num_rows) if idx != i], number_of_shots)

        for j in random_indices:
            example_image_path = all_data[0][j]
            example_image_base64 = load_image(example_image_path)
            examples.append({"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{example_image_base64}", "detail": "high"}})
            examples.append({"type": "text", "text": f'{{"prediction": "{all_data.at[j, 1]}"}}' })


        result = await get_image_informations(session, {"image": image_base64, "examples": examples})
        prediction = result["choices"][0]['message']['content']
        all_data_results.at[i, f"# of Shots {number_of_shots}"] = json.loads(prediction)['prediction']

    except Exception as e:
        print(f"Error processing {all_data[0][i]}: {str(e)}")
        all_data_results.at[i, f"# of Shots {number_of_shots}"] = 'NA'


async def process_images_for_shots(number_of_shots, all_data_results):
    async with aiohttp.ClientSession() as session:
        tasks = []
        for i in range(len(all_data)):#range(0, min(samples_to_run, len(all_data))):
            task = asyncio.ensure_future(process_image(session, i, number_of_shots, all_data_results))
            tasks.append(task)
        await asyncio.gather(*tasks)

async def main():
    all_data_results = all_data.copy(deep=True)
    all_data_results.columns = all_data_results.columns.map(str)

    for number_of_shots in [0, 1, 2, 4, 8]:
        await process_images_for_shots(number_of_shots, all_data_results)
    all_data_results.to_csv(base_directory.split("/")[-1] + ".csv")
    # print(all_data_results.iloc[9:16])
    print("done")

asyncio.run(main())

# 11. Bean Leaf Lesions Classification

In [ ]:
import os
import json
import pandas as pd
import numpy as np
import base64
import requests
import asyncio
import aiohttp
import nest_asyncio
import time

nest_asyncio.apply()
global all_data_results
import random
import os
import pandas as pd
from sklearn.utils import shuffle
from fuzzywuzzy import process

# Set the base directory
base_directory = 'DIRECTORY/AgEval-datasets/bean-leaf-lesions-classification/train'

# FOR API
max_requests_per_half_minute = 100

total_samples_to_check=100
# Define the list of expected class labels
expected_classes = ['Angular Leaf Spot', 'Bean Rust', 'Healthy' ]

num_instances_per_class = int(total_samples_to_check/len(expected_classes))
# Create empty lists to store file paths and labels
file_paths = []
labels = []

# Loop through all subdirectories in the base directory
for subdir in os.listdir(base_directory):
    if subdir==".DS_Store":
        continue
    # if subdir in expected_classes:
    subdir_path = os.path.join(base_directory, subdir)
    # Loop through all files in the subdirectory
    for filename in os.listdir(subdir_path):
        if filename.endswith('.jpg') or filename.endswith('.png'):
            # Append the file path and label to the respective lists
            file_paths.append(os.path.join(subdir_path, filename))
            labels.append(subdir)

# Create a dataframe from the lists
data = pd.DataFrame({0: file_paths, 1: labels})

# Get the unique classes
unique_classes = data[1].unique()

# Create a new dataframe with sampled data
sampled_data = pd.DataFrame(columns=[0, 1])

# Sample uniformly from each class
for cls in unique_classes:
    class_data = data[data[1] == cls].sample(n=num_instances_per_class, random_state=42)
    sampled_data = pd.concat([sampled_data, class_data], ignore_index=True)

# Shuffle the sampled data
all_data = shuffle(sampled_data, random_state=42).reset_index(drop=True)



def load_image(image_path: str) -> str:
    """Load image from file and encode it as base64."""
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

api_key = os.getenv("OPENAI_API_KEY")
model ="gpt-4o-2024-05-13" #"gpt-4-turbo"
url = "https://api.openai.com/v1/chat/completions"
headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {api_key}"
}

vision_prompt = f"""
Given the image, identify the class. Use the following list of possible classes for your prediction It should be one of the : {expected_classes}. Be attentive to subtle details as some classes may appear similar. Provide your answer in the following JSON format:
{{"prediction": "class_name"}}
Replace "class_name" with the appropriate class from the list above based on your analysis of the image.
"""


request_times = []

async def get_image_informations(session, inputs: dict) -> dict:
    """Invoke model with image and prompt."""
    global request_times
    
    while True:
        current_time = time.time()
        
        # Remove old request times outside the current time window
        while request_times and request_times[0] <= current_time - 30:
            request_times.pop(0)
        
        if len(request_times) < max_requests_per_half_minute:
            request_times.append(current_time)
            break
        else:
            # If the rate limit is exceeded, wait for a short duration before checking again
            await asyncio.sleep(1)
    
    
    payload = {
        "model": model,
        "response_format":{ "type": "json_object" },
        "messages": [
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": vision_prompt},
                    *inputs['examples'],
                    {
                        "type": "image_url",
                        "image_url": {
                            "url": f"data:image/jpeg;base64,{inputs['image']}",
                            "detail": "high"
                        }
                    }
                ]
            }
        ],
        "max_tokens": 4096
    }
    async with session.post(url, headers=headers, json=payload) as response:
        result = await response.json()
        print(result)
        if "choices" in result and result["choices"]:
            return result
        else:
            raise Exception("Unexpected API response format")

async def process_image(session, i, number_of_shots, all_data_results):
    try:
        image_path = all_data[0][i]
        image_base64 = load_image(image_path)
        examples = []

        # Get the number of rows in all_data
        num_rows = len(all_data)

        # Select random indices from the dataframe
        random_indices = random.sample([idx for idx in range(num_rows) if idx != i], number_of_shots)

        for j in random_indices:
            example_image_path = all_data[0][j]
            example_image_base64 = load_image(example_image_path)
            examples.append({"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{example_image_base64}", "detail": "high"}})
            examples.append({"type": "text", "text": f'{{"prediction": "{all_data.at[j, 1]}"}}' })


        result = await get_image_informations(session, {"image": image_base64, "examples": examples})
        prediction = result["choices"][0]['message']['content']
        all_data_results.at[i, f"# of Shots {number_of_shots}"] = json.loads(prediction)['prediction']

    except Exception as e:
        print(f"Error processing {all_data[0][i]}: {str(e)}")
        all_data_results.at[i, f"# of Shots {number_of_shots}"] = 'NA'


async def process_images_for_shots(number_of_shots, all_data_results):
    async with aiohttp.ClientSession() as session:
        tasks = []
        for i in range(len(all_data)):#range(0, min(samples_to_run, len(all_data))):
            task = asyncio.ensure_future(process_image(session, i, number_of_shots, all_data_results))
            tasks.append(task)
        await asyncio.gather(*tasks)

async def main():
    all_data_results = all_data.copy(deep=True)
    all_data_results.columns = all_data_results.columns.map(str)

    for number_of_shots in [0, 1, 2, 4, 8]:
        await process_images_for_shots(number_of_shots, all_data_results)
    all_data_results.to_csv(base_directory.split("/")[-1] + ".csv")
    # print(all_data_results.iloc[9:16])
    print("done")

asyncio.run(main())

# 12. DeeWeeds


In [ ]:
import os
import json
import pandas as pd
import numpy as np
import base64
import requests
import asyncio
import aiohttp
import nest_asyncio
import time
import random
from sklearn.utils import shuffle

nest_asyncio.apply()
global all_data_results

# Set the base directory
base_directory = 'DIRECTORY/AgEval-datasets/deepweeds'

# FOR API
max_requests_per_half_minute = 100

total_samples_to_check = 100

# Load the labels CSV file
labels_df = pd.read_csv(os.path.join(base_directory, 'labels', 'labels.csv'))

# Get the unique classes (species)
expected_classes = ['Chinee apple', 'Lantana', 'Negative', 'Snake weed', 'Siam weed', 'Prickly acacia', 'Parthenium', 'Rubber vine', 'Parkinsonia']

#labels_df['Species'].unique().tolist()

num_instances_per_class = int(total_samples_to_check / len(expected_classes))

# Create a dataframe with file paths and labels
data = pd.DataFrame({
    'file_path': labels_df['Filename'].apply(lambda x: os.path.join(base_directory, 'images', x)),
    'label': labels_df['Species']
})

# Sample uniformly from each class
sampled_data = pd.DataFrame(columns=['file_path', 'label'])
for cls in expected_classes:
    class_data = data[data['label'] == cls].sample(n=num_instances_per_class, random_state=42)
    sampled_data = pd.concat([sampled_data, class_data], ignore_index=True)

# Shuffle the sampled data
all_data = shuffle(sampled_data, random_state=42).reset_index(drop=True)

def load_image(image_path: str) -> str:
    """Load image from file and encode it as base64."""
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

api_key = os.getenv("OPENAI_API_KEY")
model = "gpt-4o-2024-05-13"  # "gpt-4-turbo"
url = "https://api.openai.com/v1/chat/completions"
headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {api_key}"
}

vision_prompt = f"""
Given the image, identify the species of weed. Use the following list of possible classes for your prediction: {expected_classes}. Be attentive to subtle details as some classes may appear similar. Provide your answer in the following JSON format:
{{"prediction": "species_name"}}
Replace "species_name" with the appropriate species from the list above based on your analysis of the image.
"""

request_times = []

async def get_image_informations(session, inputs: dict) -> dict:
    """Invoke model with image and prompt."""
    global request_times
    
    while True:
        current_time = time.time()
        
        # Remove old request times outside the current time window
        while request_times and request_times[0] <= current_time - 30:
            request_times.pop(0)
        
        if len(request_times) < max_requests_per_half_minute:
            request_times.append(current_time)
            break
        else:
            # If the rate limit is exceeded, wait for a short duration before checking again
            await asyncio.sleep(1)
    
    payload = {
        "model": model,
        "response_format": {"type": "json_object"},
        "messages": [
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": vision_prompt},
                    *inputs['examples'],
                    {
                        "type": "image_url",
                        "image_url": {
                            "url": f"data:image/jpeg;base64,{inputs['image']}",
                            "detail": "high"
                        }
                    }
                ]
            }
        ],
        "max_tokens": 4096
    }
    async with session.post(url, headers=headers, json=payload) as response:
        result = await response.json()
        print(result)
        if "choices" in result and result["choices"]:
            return result
        else:
            raise Exception("Unexpected API response format")

async def process_image(session, i, number_of_shots, all_data_results):
    try:
        image_path = all_data['file_path'][i]
        image_base64 = load_image(image_path)
        examples = []

        # Get the number of rows in all_data
        num_rows = len(all_data)

        # Select random indices from the dataframe
        random_indices = random.sample([idx for idx in range(num_rows) if idx != i], number_of_shots)

        for j in random_indices:
            example_image_path = all_data['file_path'][j]
            example_image_base64 = load_image(example_image_path)
            examples.append({"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{example_image_base64}", "detail": "high"}})
            examples.append({"type": "text", "text": f'{{"prediction": "{all_data.at[j, "label"]}"}}' })

        result = await get_image_informations(session, {"image": image_base64, "examples": examples})
        prediction = result["choices"][0]['message']['content']
        all_data_results.at[i, f"# of Shots {number_of_shots}"] = json.loads(prediction)['prediction']

    except Exception as e:
        print(f"Error processing {all_data['file_path'][i]}: {str(e)}")
        all_data_results.at[i, f"# of Shots {number_of_shots}"] = 'NA'

async def process_images_for_shots(number_of_shots, all_data_results):
    async with aiohttp.ClientSession() as session:
        tasks = []
        for i in range(len(all_data)):
            task = asyncio.ensure_future(process_image(session, i, number_of_shots, all_data_results))
            tasks.append(task)
        await asyncio.gather(*tasks)

async def main():
    all_data_results = all_data.copy(deep=True)

    for number_of_shots in [0, 1, 2, 4, 8]:
        await process_images_for_shots(number_of_shots, all_data_results)
    all_data_results.to_csv(base_directory.split("/")[-1] + ".csv")
    print("done")

asyncio.run(main())

# IP102

In [ ]:
import os
import json
import pandas as pd
import numpy as np
import base64
import requests
import asyncio
import aiohttp
import nest_asyncio
import time
import random
from sklearn.utils import shuffle

nest_asyncio.apply()
global all_data_results

# Set the base directory
base_directory = 'DIRECTORY/AgEval-datasets/ip02-dataset/classification/train'
classes_file = 'DIRECTORY/AgEval-datasets/ip02-dataset/classes.txt'

# FOR API
max_requests_per_half_minute = 100

total_samples_to_check = 102*3

# Read the classes from the txt file
def read_classes(file_path):
    classes = {}
    with open(file_path, 'r') as f:
        for line in f:
            parts = line.strip().split(maxsplit=1)
            if len(parts) == 2:
                class_id, class_name = parts
                classes[int(class_id) - 1] = class_name.strip()  # Subtract 1 to match folder names
    return classes

expected_classes = read_classes(classes_file)

num_instances_per_class = int(total_samples_to_check / len(expected_classes))

# Create empty lists to store file paths and labels
file_paths = []
labels = []

# Loop through all subdirectories in the base directory
for subdir in os.listdir(base_directory):
    if subdir == ".DS_Store":
        continue
    subdir_path = os.path.join(base_directory, subdir)
    if os.path.isdir(subdir_path):
        # Loop through all files in the subdirectory
        for filename in os.listdir(subdir_path):
            if filename.endswith('.jpg') or filename.endswith('.png'):
                # Append the file path and label to the respective lists
                file_paths.append(os.path.join(subdir_path, filename))
                labels.append(expected_classes[int(subdir)])

# Create a dataframe from the lists
data = pd.DataFrame({0: file_paths, 1: labels})

# Get the unique classes
unique_classes = data[1].unique()

# Create a new dataframe with sampled data
sampled_data = pd.DataFrame(columns=[0, 1])

# Sample uniformly from each class
for cls in unique_classes:
    class_data = data[data[1] == cls].sample(n=min(num_instances_per_class, len(data[data[1] == cls])), random_state=42)
    sampled_data = pd.concat([sampled_data, class_data], ignore_index=True)

# Shuffle the sampled data
all_data = shuffle(sampled_data, random_state=42).reset_index(drop=True)

def load_image(image_path: str) -> str:
    """Load image from file and encode it as base64."""
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

api_key = os.getenv("OPENAI_API_KEY")
model = "gpt-4o-2024-05-13"  # "gpt-4-turbo"
url = "https://api.openai.com/v1/chat/completions"
headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {api_key}"
}

vision_prompt = f"""
Given the image, identify the class. Use the following list of possible classes for your prediction: {list(expected_classes.values())}. Be attentive to subtle details as some classes may appear similar. Provide your answer in the following JSON format:
{{"prediction": "class_name"}}
Replace "class_name" with the appropriate class from the list above based on your analysis of the image.
"""

request_times = []

async def get_image_informations(session, inputs: dict) -> dict:
    """Invoke model with image and prompt."""
    global request_times
    
    while True:
        current_time = time.time()
        
        # Remove old request times outside the current time window
        while request_times and request_times[0] <= current_time - 30:
            request_times.pop(0)
        
        if len(request_times) < max_requests_per_half_minute:
            request_times.append(current_time)
            break
        else:
            # If the rate limit is exceeded, wait for a short duration before checking again
            await asyncio.sleep(1)
    
    payload = {
        "model": model,
        "response_format":{ "type": "json_object" },
        "messages": [
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": vision_prompt},
                    *inputs['examples'],
                    {
                        "type": "image_url",
                        "image_url": {
                            "url": f"data:image/jpeg;base64,{inputs['image']}",
                            "detail": "high"
                        }
                    }
                ]
            }
        ],
        "max_tokens": 4096
    }
    async with session.post(url, headers=headers, json=payload) as response:
        result = await response.json()
        print(result)
        if "choices" in result and result["choices"]:
            return result
        else:
            raise Exception("Unexpected API response format")

async def process_image(session, i, number_of_shots, all_data_results):
    try:
        image_path = all_data[0][i]
        image_base64 = load_image(image_path)
        examples = []
        example_paths = []
        example_categories = []

        # Get the number of rows in all_data
        num_rows = len(all_data)

        # Select random indices from the dataframe
        random_indices = random.sample([idx for idx in range(num_rows) if idx != i], number_of_shots)

        for j in random_indices:
            example_image_path = all_data[0][j]
            example_category = all_data[1][j]
            example_image_base64 = load_image(example_image_path)
            examples.append({"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{example_image_base64}", "detail": "high"}})
            examples.append({"type": "text", "text": f'{{"prediction": "{example_category}"}}' })
            example_paths.append(example_image_path)
            example_categories.append(example_category)

        result = await get_image_informations(session, {"image": image_base64, "examples": examples})
        prediction = result["choices"][0]['message']['content']
        all_data_results.at[i, f"# of Shots {number_of_shots}"] = json.loads(prediction)['prediction']
        all_data_results.at[i, f"Example Paths {number_of_shots}"] = json.dumps(example_paths)
        all_data_results.at[i, f"Example Categories {number_of_shots}"] = json.dumps(example_categories)

    except Exception as e:
        print(f"Error processing {all_data[0][i]}: {str(e)}")
        all_data_results.at[i, f"# of Shots {number_of_shots}"] = 'NA'
        all_data_results.at[i, f"Example Paths {number_of_shots}"] = 'NA'
        all_data_results.at[i, f"Example Categories {number_of_shots}"] = 'NA'

async def process_images_for_shots(number_of_shots, all_data_results):
    async with aiohttp.ClientSession() as session:
        tasks = []
        for i in range(len(all_data)):
            task = asyncio.ensure_future(process_image(session, i, number_of_shots, all_data_results))
            tasks.append(task)
        await asyncio.gather(*tasks)

async def main():
    all_data_results = all_data.copy(deep=True)
    all_data_results.columns = all_data_results.columns.map(str)

    for number_of_shots in [0,1, 2, 4, 8]:
        await process_images_for_shots(number_of_shots, all_data_results)
    all_data_results.to_csv("ip02-dataset-results.csv")
    print("done")

asyncio.run(main())

# Yellow Rust 19


In [ ]:
import os
import json
import pandas as pd
import numpy as np
import base64
import requests
import asyncio
import aiohttp
import nest_asyncio
import time

nest_asyncio.apply()
global all_data_results
import random
import os
import pandas as pd
from sklearn.utils import shuffle
from fuzzywuzzy import process

# Set the base directory
base_directory = 'DIRECTORY/AgEval-datasets/yellowrust19-yellow-rust-disease-in-wheat/YELLOW-RUST-19/YELLOW-RUST-19'

# FOR API
max_requests_per_half_minute = 100

total_samples_to_check=100
# Define the list of expected class labels
expected_classes = ['Moderately Resistant (MR)', 'Moderately Susceptible (MS)', 'MRMS', 'No disease (0)', 'Resistant (R)', 'Susceptible (S)' ]

num_instances_per_class = int(total_samples_to_check/len(expected_classes))
# Create empty lists to store file paths and labels
file_paths = []
labels = []

# Loop through all subdirectories in the base directory
for subdir in os.listdir(base_directory):
    if subdir==".DS_Store":
        continue
    # if subdir in expected_classes:
    subdir_path = os.path.join(base_directory, subdir)
    # Loop through all files in the subdirectory
    for filename in os.listdir(subdir_path):
        if filename.endswith('.jpg') or filename.endswith('.png'):
            # Append the file path and label to the respective lists
            file_paths.append(os.path.join(subdir_path, filename))
            labels.append(subdir)

# Create a dataframe from the lists
data = pd.DataFrame({0: file_paths, 1: labels})

# Get the unique classes
unique_classes = data[1].unique()

# Create a new dataframe with sampled data
sampled_data = pd.DataFrame(columns=[0, 1])

# Sample uniformly from each class
for cls in unique_classes:
    class_data = data[data[1] == cls].sample(n=num_instances_per_class, random_state=42)
    sampled_data = pd.concat([sampled_data, class_data], ignore_index=True)

# Shuffle the sampled data
all_data = shuffle(sampled_data, random_state=42).reset_index(drop=True)



def load_image(image_path: str) -> str:
    """Load image from file and encode it as base64."""
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

api_key = os.getenv("OPENAI_API_KEY")
model ="gpt-4o-2024-05-13" #"gpt-4-turbo"
url = "https://api.openai.com/v1/chat/completions"
headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {api_key}"
}

vision_prompt = f"""
Given the image, identify the class. Use the following list of possible classes for your prediction It should be one of the : {expected_classes}. Be attentive to subtle details as some classes may appear similar. Provide your answer in the following JSON format:
{{"prediction": "class_name"}}
Replace "class_name" with the appropriate class from the list above based on your analysis of the image.
Please note that: 
No disease (0: No signs of infection),
Resistant (R: Minor signs of infection),
Moderately Resistant (MR: Small and medium signs of infection),
Moderately Resistant-Moderately Susceptible (MRMS),
Moderately Susceptible (MS: Medium signs of infection) and
Susceptible (S: Major signs of infection)
The labels should be entered exactly as they are in the list above i.e., {expected_classes}
"""


request_times = []

async def get_image_informations(session, inputs: dict) -> dict:
    """Invoke model with image and prompt."""
    global request_times
    
    while True:
        current_time = time.time()
        
        # Remove old request times outside the current time window
        while request_times and request_times[0] <= current_time - 30:
            request_times.pop(0)
        
        if len(request_times) < max_requests_per_half_minute:
            request_times.append(current_time)
            break
        else:
            # If the rate limit is exceeded, wait for a short duration before checking again
            await asyncio.sleep(1)
    
    
    payload = {
        "model": model,
        "response_format":{ "type": "json_object" },
        "messages": [
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": vision_prompt},
                    *inputs['examples'],
                    {
                        "type": "image_url",
                        "image_url": {
                            "url": f"data:image/jpeg;base64,{inputs['image']}",
                            "detail": "high"
                        }
                    }
                ]
            }
        ],
        "max_tokens": 4096
    }
    async with session.post(url, headers=headers, json=payload) as response:
        result = await response.json()
        print(result)
        if "choices" in result and result["choices"]:
            return result
        else:
            raise Exception("Unexpected API response format")

async def process_image(session, i, number_of_shots, all_data_results):
    try:
        image_path = all_data[0][i]
        image_base64 = load_image(image_path)
        examples = []

        # Get the number of rows in all_data
        num_rows = len(all_data)

        # Select random indices from the dataframe
        random_indices = random.sample([idx for idx in range(num_rows) if idx != i], number_of_shots)

        for j in random_indices:
            example_image_path = all_data[0][j]
            example_image_base64 = load_image(example_image_path)
            examples.append({"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{example_image_base64}", "detail": "high"}})
            examples.append({"type": "text", "text": f'{{"prediction": "{all_data.at[j, 1]}"}}' })


        result = await get_image_informations(session, {"image": image_base64, "examples": examples})
        prediction = result["choices"][0]['message']['content']
        all_data_results.at[i, f"# of Shots {number_of_shots}"] = json.loads(prediction)['prediction']

    except Exception as e:
        print(f"Error processing {all_data[0][i]}: {str(e)}")
        all_data_results.at[i, f"# of Shots {number_of_shots}"] = 'NA'


async def process_images_for_shots(number_of_shots, all_data_results):
    async with aiohttp.ClientSession() as session:
        tasks = []
        for i in range(len(all_data)):#range(0, min(samples_to_run, len(all_data))):
            task = asyncio.ensure_future(process_image(session, i, number_of_shots, all_data_results))
            tasks.append(task)
        await asyncio.gather(*tasks)

async def main():
    all_data_results = all_data.copy(deep=True)
    all_data_results.columns = all_data_results.columns.map(str)

    for number_of_shots in [0, 1, 2, 4, 8]:
        await process_images_for_shots(number_of_shots, all_data_results)
    all_data_results.to_csv(base_directory.split("/")[-1] + ".csv")
    # print(all_data_results.iloc[9:16])
    print("done")

asyncio.run(main())

# Fusarium

In [ ]:
import os
import json
import pandas as pd
import numpy as np
import base64
import requests
import asyncio
import aiohttp
import nest_asyncio
import time

nest_asyncio.apply()
global all_data_results
import random
import os
import pandas as pd
from sklearn.utils import shuffle
from fuzzywuzzy import process

# Set the base directory
base_directory = 'DIRECTORY/AgEval-datasets/FUSARIUM-22/dataset_raw'
# FOR API
max_requests_per_half_minute = 100

total_samples_to_check=100
# Define the list of expected class labels
expected_classes = ['Highly Resistant', 'Highly Susceptible', 'Moderately Resistant', 'Resistant', 'Susceptible' ]

num_instances_per_class = int(total_samples_to_check/len(expected_classes))
# Create empty lists to store file paths and labels
file_paths = []
labels = []

# Loop through all subdirectories in the base directory
for subdir in os.listdir(base_directory):
    if subdir==".DS_Store":
        continue
    # if subdir in expected_classes:
    subdir_path = os.path.join(base_directory, subdir)
    # Loop through all files in the subdirectory
    for filename in os.listdir(subdir_path):
        if filename.endswith('.jpg') or filename.endswith('.png'):
            # Append the file path and label to the respective lists
            file_paths.append(os.path.join(subdir_path, filename))
            labels.append(subdir)

# Create a dataframe from the lists
data = pd.DataFrame({0: file_paths, 1: labels})

# Get the unique classes
unique_classes = data[1].unique()

# Create a new dataframe with sampled data
sampled_data = pd.DataFrame(columns=[0, 1])

# Sample uniformly from each class
for cls in unique_classes:
    class_data = data[data[1] == cls].sample(n=num_instances_per_class, random_state=42)
    sampled_data = pd.concat([sampled_data, class_data], ignore_index=True)

# Shuffle the sampled data
all_data = shuffle(sampled_data, random_state=42).reset_index(drop=True)



def load_image(image_path: str) -> str:
    """Load image from file and encode it as base64."""
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

api_key = os.getenv("OPENAI_API_KEY")
model ="gpt-4o-2024-05-13" #"gpt-4-turbo"
url = "https://api.openai.com/v1/chat/completions"
headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {api_key}"
}

vision_prompt = f"""
Given the image, identify the class. Use the following list of possible classes for your prediction It should be one of the : {expected_classes}. Be attentive to subtle details as some classes may appear similar. Provide your answer in the following JSON format:
{{"prediction": "class_name"}}
Replace "class_name" with the appropriate class from the list above based on your analysis of the image.
Please note that: 
Severity level images from the dataset:
1: Highly Resistant (HR): The plant has been wilted by 0%-10%,
3: Resistant (R): The plant has been wilted by 11%-20%,
5: Moderately Resistant/ Tolerant (MR): The plant has been wilted by 21%-30%,
7: Susceptible (S): The plant has been wilted by 31%-50%,
9: Highly Susceptible (HS): The plant has been wilted by more than 51%
The labels should be entered exactly as they are in the list above i.e., {expected_classes}
"""


request_times = []

async def get_image_informations(session, inputs: dict) -> dict:
    """Invoke model with image and prompt."""
    global request_times
    
    while True:
        current_time = time.time()
        
        # Remove old request times outside the current time window
        while request_times and request_times[0] <= current_time - 30:
            request_times.pop(0)
        
        if len(request_times) < max_requests_per_half_minute:
            request_times.append(current_time)
            break
        else:
            # If the rate limit is exceeded, wait for a short duration before checking again
            await asyncio.sleep(1)
    
    
    payload = {
        "model": model,
        "response_format":{ "type": "json_object" },
        "messages": [
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": vision_prompt},
                    *inputs['examples'],
                    {
                        "type": "image_url",
                        "image_url": {
                            "url": f"data:image/jpeg;base64,{inputs['image']}",
                            "detail": "high"
                        }
                    }
                ]
            }
        ],
        "max_tokens": 4096
    }
    async with session.post(url, headers=headers, json=payload) as response:
        result = await response.json()
        print(result)
        if "choices" in result and result["choices"]:
            return result
        else:
            raise Exception("Unexpected API response format")

async def process_image(session, i, number_of_shots, all_data_results):
    try:
        image_path = all_data[0][i]
        image_base64 = load_image(image_path)
        examples = []

        # Get the number of rows in all_data
        num_rows = len(all_data)

        # Select random indices from the dataframe
        random_indices = random.sample([idx for idx in range(num_rows) if idx != i], number_of_shots)

        for j in random_indices:
            example_image_path = all_data[0][j]
            example_image_base64 = load_image(example_image_path)
            examples.append({"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{example_image_base64}", "detail": "high"}})
            examples.append({"type": "text", "text": f'{{"prediction": "{all_data.at[j, 1]}"}}' })


        result = await get_image_informations(session, {"image": image_base64, "examples": examples})
        prediction = result["choices"][0]['message']['content']
        all_data_results.at[i, f"# of Shots {number_of_shots}"] = json.loads(prediction)['prediction']

    except Exception as e:
        print(f"Error processing {all_data[0][i]}: {str(e)}")
        all_data_results.at[i, f"# of Shots {number_of_shots}"] = 'NA'


async def process_images_for_shots(number_of_shots, all_data_results):
    async with aiohttp.ClientSession() as session:
        tasks = []
        for i in range(len(all_data)):#range(0, min(samples_to_run, len(all_data))):
            task = asyncio.ensure_future(process_image(session, i, number_of_shots, all_data_results))
            tasks.append(task)
        await asyncio.gather(*tasks)

async def main():
    all_data_results = all_data.copy(deep=True)
    all_data_results.columns = all_data_results.columns.map(str)

    for number_of_shots in [0, 1, 2, 4, 8]:
        await process_images_for_shots(number_of_shots, all_data_results)
    all_data_results.to_csv("Furasium.csv")
    # print(all_data_results.iloc[9:16])
    print("done")

asyncio.run(main())

# Severity based rice disease

In [ ]:
import os
import json
import pandas as pd
import numpy as np
import base64
import requests
import asyncio
import aiohttp
import nest_asyncio
import time

nest_asyncio.apply()
global all_data_results
import random
import os
import pandas as pd
from sklearn.utils import shuffle
from fuzzywuzzy import process

# Set the base directory
base_directory = 'DIRECTORY/AgEval-datasets/severity-based-rice-disease/train'
# FOR API
max_requests_per_half_minute = 100

total_samples_to_check=100

# Define the list of expected class labels
expected_classes = ['Healthy', 'Mild Bacterial Blight', 'Mild Blast', 'Mild Brownspot', 'Mild Tungro', 'Severe Bacterial Blight', 'Severe Blast', 'Severe Brownspot', 'Severe Tungro' ]

num_instances_per_class = int(total_samples_to_check/len(expected_classes))
# Create empty lists to store file paths and labels
file_paths = []
labels = []

# Loop through all subdirectories in the base directory
for subdir in os.listdir(base_directory):
    if subdir==".DS_Store":
        continue
    # if subdir in expected_classes:
    subdir_path = os.path.join(base_directory, subdir)
    # Loop through all files in the subdirectory
    for filename in os.listdir(subdir_path):
        if filename.endswith('.jpg') or filename.endswith('.png') or filename.endswith('.JPG') or filename.endswith('.PNG'):
            # Append the file path and label to the respective lists
            file_paths.append(os.path.join(subdir_path, filename))
            labels.append(subdir)

# Create a dataframe from the lists
data = pd.DataFrame({0: file_paths, 1: labels})

# Get the unique classes
unique_classes = data[1].unique()

# Create a new dataframe with sampled data
sampled_data = pd.DataFrame(columns=[0, 1])

# Sample uniformly from each class
for cls in unique_classes:
    class_data = data[data[1] == cls].sample(n=num_instances_per_class, random_state=42)
    sampled_data = pd.concat([sampled_data, class_data], ignore_index=True)

# Shuffle the sampled data
all_data = shuffle(sampled_data, random_state=42).reset_index(drop=True)



def load_image(image_path: str) -> str:
    """Load image from file and encode it as base64."""
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

api_key = os.getenv("OPENAI_API_KEY")
model ="gpt-4o-2024-05-13" #"gpt-4-turbo"
url = "https://api.openai.com/v1/chat/completions"
headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {api_key}"
}

vision_prompt = f"""
Given the image, identify the class. Use the following list of possible classes for your prediction It should be one of the : {expected_classes}. Be attentive to subtle details as some classes may appear similar. Provide your answer in the following JSON format:
{{"prediction": "class_name"}}
Replace "class_name" with the appropriate class from the list above based on your analysis of the image.
The labels should be entered exactly as they are in the list above i.e., {expected_classes}
"""


request_times = []

async def get_image_informations(session, inputs: dict) -> dict:
    """Invoke model with image and prompt."""
    global request_times
    
    while True:
        current_time = time.time()
        
        # Remove old request times outside the current time window
        while request_times and request_times[0] <= current_time - 30:
            request_times.pop(0)
        
        if len(request_times) < max_requests_per_half_minute:
            request_times.append(current_time)
            break
        else:
            # If the rate limit is exceeded, wait for a short duration before checking again
            await asyncio.sleep(1)
    
    
    payload = {
        "model": model,
        "response_format":{ "type": "json_object" },
        "messages": [
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": vision_prompt},
                    *inputs['examples'],
                    {
                        "type": "image_url",
                        "image_url": {
                            "url": f"data:image/jpeg;base64,{inputs['image']}",
                            "detail": "high"
                        }
                    }
                ]
            }
        ],
        "max_tokens": 4096
    }
    async with session.post(url, headers=headers, json=payload) as response:
        result = await response.json()
        print(result)
        if "choices" in result and result["choices"]:
            return result
        else:
            raise Exception("Unexpected API response format")

async def process_image(session, i, number_of_shots, all_data_results):
    try:
        image_path = all_data[0][i]
        image_base64 = load_image(image_path)
        examples = []

        # Get the number of rows in all_data
        num_rows = len(all_data)

        # Select random indices from the dataframe
        random_indices = random.sample([idx for idx in range(num_rows) if idx != i], number_of_shots)

        for j in random_indices:
            example_image_path = all_data[0][j]
            example_image_base64 = load_image(example_image_path)
            examples.append({"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{example_image_base64}", "detail": "high"}})
            examples.append({"type": "text", "text": f'{{"prediction": "{all_data.at[j, 1]}"}}' })


        result = await get_image_informations(session, {"image": image_base64, "examples": examples})
        prediction = result["choices"][0]['message']['content']
        all_data_results.at[i, f"# of Shots {number_of_shots}"] = json.loads(prediction)['prediction']

    except Exception as e:
        print(f"Error processing {all_data[0][i]}: {str(e)}")
        all_data_results.at[i, f"# of Shots {number_of_shots}"] = 'NA'


async def process_images_for_shots(number_of_shots, all_data_results):
    async with aiohttp.ClientSession() as session:
        tasks = []
        for i in range(len(all_data)):#range(0, min(samples_to_run, len(all_data))):
            task = asyncio.ensure_future(process_image(session, i, number_of_shots, all_data_results))
            tasks.append(task)
        await asyncio.gather(*tasks)

async def main():
    all_data_results = all_data.copy(deep=True)
    all_data_results.columns = all_data_results.columns.map(str)

    for number_of_shots in [0, 1, 2, 4, 8]:
        await process_images_for_shots(number_of_shots, all_data_results)
    all_data_results.to_csv("SBRD.csv")
    # print(all_data_results.iloc[9:16])
    print("done")

asyncio.run(main())

In [ ]:
import os
import json
import pandas as pd
import numpy as np
import base64
import requests
import asyncio
import aiohttp
import nest_asyncio
import time
from anthropic import Anthropic
import argparse
from PIL import Image
nest_asyncio.apply()
global all_data_results
import random
import os
import pandas as pd
from sklearn.utils import shuffle
from fuzzywuzzy import process
import io
# Set the base directory
base_directory = 'DIRECTORY/AgEval-datasets/severity-based-rice-disease/train'
model_type = 'claude'
# FOR API
max_requests_per_half_minute = 5

total_samples_to_check = 10

# Define the list of expected class labels
expected_classes = ['Healthy', 'Mild Bacterial Blight', 'Mild Blast', 'Mild Brownspot', 'Mild Tungro', 'Severe Bacterial Blight', 'Severe Blast', 'Severe Brownspot', 'Severe Tungro']

num_instances_per_class = int(total_samples_to_check/len(expected_classes))
# Create empty lists to store file paths and labels
file_paths = []
labels = []

# Loop through all subdirectories in the base directory
for subdir in os.listdir(base_directory):
    if subdir == ".DS_Store":
        continue
    subdir_path = os.path.join(base_directory, subdir)
    # Loop through all files in the subdirectory
    for filename in os.listdir(subdir_path):
        if filename.endswith('.jpg') or filename.endswith('.png') or filename.endswith('.JPG') or filename.endswith('.PNG'):
            # Append the file path and label to the respective lists
            file_paths.append(os.path.join(subdir_path, filename))
            labels.append(subdir)

# Create a dataframe from the lists
data = pd.DataFrame({0: file_paths, 1: labels})

# Get the unique classes
unique_classes = data[1].unique()

# Create a new dataframe with sampled data
sampled_data = pd.DataFrame(columns=[0, 1])

# Sample uniformly from each class
for cls in unique_classes:
    class_data = data[data[1] == cls].sample(n=num_instances_per_class, random_state=42)
    sampled_data = pd.concat([sampled_data, class_data], ignore_index=True)

# Shuffle the sampled data
all_data = shuffle(sampled_data, random_state=42).reset_index(drop=True)

def load_image(image_path: str) -> str:
    """
    Load image from file, convert to JPEG, and encode as base64.
    """
    try:
        with Image.open(image_path) as img:
            # Convert image to RGB mode if it's not
            if img.mode != 'RGB':
                img = img.convert('RGB')
            
            # Save image to a bytes buffer as JPEG
            buffer = io.BytesIO()
            img.save(buffer, format="JPEG", quality=85)  # You can adjust quality as needed
            
            # Encode the JPEG image as base64
            return base64.b64encode(buffer.getvalue()).decode('utf-8')
    except Exception as e:
        print(f"Error processing image {image_path}: {str(e)}")
        return None
    
    
vision_prompt = f"""
Given the image, identify the class. Use the following list of possible classes for your prediction It should be one of the : {expected_classes}. Be attentive to subtle details as some classes may appear similar. Provide your answer in the following JSON format:
{{"prediction": "class_name"}}
Replace "class_name" with the appropriate class from the list above based on your analysis of the image.
The labels should be entered exactly as they are in the list above i.e., {expected_classes}.
The response should start with {{ and contain only a JSON object (as specified above) and no other text.
"""

request_times = []

# Existing GPT-4 Vision function (unchanged)
async def get_image_informations_gpt(session, inputs: dict) -> dict:
    """Invoke GPT-4 Vision model with image and prompt."""
    global request_times
    
    while True:
        current_time = time.time()
        
        # Remove old request times outside the current time window
        while request_times and request_times[0] <= current_time - 30:
            request_times.pop(0)
        
        if len(request_times) < max_requests_per_half_minute:
            request_times.append(current_time)
            break
        else:
            # If the rate limit is exceeded, wait for a short duration before checking again
            await asyncio.sleep(1)
    
    api_key = os.getenv("OPENAI_API_KEY")
    model = "gpt-4o-2024-05-13"  # or "gpt-4-turbo"
    url = "https://api.openai.com/v1/chat/completions"
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {api_key}"
    }
    
    payload = {
        "model": model,
        "response_format": {"type": "json_object"},
        "messages": [
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": vision_prompt},
                    *inputs['examples'],
                    {
                        "type": "image_url",
                        "image_url": {
                            "url": f"data:image/jpeg;base64,{inputs['image']}",
                            "detail": "high"
                        }
                    }
                ]
            }
        ],
        "max_tokens": 4096
    }
    async with session.post(url, headers=headers, json=payload) as response:
        result = await response.json()
        print(result)
        if "choices" in result and result["choices"]:
            return result["choices"][0]['message']['content']
        else:
            raise Exception("Unexpected API response format")

# New function for Claude 3.5 Sonnet
async def get_image_informations_claude(inputs: dict) -> str:
    """Invoke Claude 3.5 Sonnet model with image and prompt."""
    anthropic_api_key = os.getenv("ANTHROPIC_API_KEY")
    anthropic_client = Anthropic(api_key=anthropic_api_key)
    
    def get_media_type(img_data):
        if img_data.startswith('data:image/jpeg;base64,'):
            return "image/jpeg"
        elif img_data.startswith('data:image/png;base64,'):
            return "image/png"
        else:
            raise ValueError(f"Unsupported image format")
    
    messages = [
        {
            "role": "user",
            "content": [
                {"type": "text", "text": vision_prompt},
                *[
                    {
                        "type": "image",
                        "source": {
                            "type": "base64",
                            "media_type": get_media_type(ex['image_url']['url']),
                            "data": ex['image_url']['url'].split(',')[1]
                        }
                    }
                    if ex['type'] == 'image_url' else ex
                    for ex in inputs['examples']
                ],
                {
                    "type": "image",
                    "source": {
                        "type": "base64",
                        "media_type": get_media_type(f"data:image/jpeg;base64,{inputs['image']}"),
                        "data": inputs['image']
                    }
                }
            ]
        }
    ]
    
    response = anthropic_client.messages.create(
        model="claude-3-5-sonnet-20240620",
        max_tokens=1000,
        temperature=0.3,
        messages=messages
    )
    return response.content[0].text

async def process_image(session, i, number_of_shots, all_data_results, model_type):
    try:
        image_path = all_data[0][i]
        image_base64 = load_image(image_path)
        examples = []

        # Get the number of rows in all_data
        num_rows = len(all_data)

        # Select random indices from the dataframe
        random_indices = random.sample([idx for idx in range(num_rows) if idx != i], number_of_shots)

        for j in random_indices:
            example_image_path = all_data[0][j]
            example_image_base64 = load_image(example_image_path)
            examples.append({"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{example_image_base64}", "detail": "high"}})
            examples.append({"type": "text", "text": f'{{"prediction": "{all_data.at[j, 1]}"}}' })

        if model_type == "gpt":
            prediction = await get_image_informations_gpt(session, {"image": image_base64, "examples": examples})
        elif model_type == "claude":
            prediction = await get_image_informations_claude({"image": image_base64, "examples": examples})
        else:
            raise ValueError(f"Unsupported model type: {model_type}")

        all_data_results.at[i, f"# of Shots {number_of_shots}"] = json.loads(prediction)['prediction']

    except Exception as e:
        print(f"Error processing {all_data[0][i]}: {str(e)}")
        all_data_results.at[i, f"# of Shots {number_of_shots}"] = 'NA'

async def process_images_for_shots(number_of_shots, all_data_results, model_type):
    if model_type == "gpt":
        async with aiohttp.ClientSession() as session:
            tasks = []
            for i in range(len(all_data)):
                task = asyncio.ensure_future(process_image(session, i, number_of_shots, all_data_results, model_type))
                tasks.append(task)
            await asyncio.gather(*tasks)
    elif model_type == "claude":
        for i in range(len(all_data)):
            await process_image(None, i, number_of_shots, all_data_results, model_type)

async def main():
    all_data_results = all_data.copy(deep=True)
    all_data_results.columns = all_data_results.columns.map(str)

    for number_of_shots in [2]:  # [0, 1, 2, 4, 8]:
        await process_images_for_shots(number_of_shots, all_data_results, model_type)
    
    # Create output directory if it doesn't exist
    output_dir = f"results_{model_type}"
    os.makedirs(output_dir, exist_ok=True)
    
    # Save results to the model-specific directory
    output_file = os.path.join(output_dir, f"SBRD_{model_type}.csv")
    all_data_results.to_csv(output_file)
    print(f"Results saved to {output_file}")

asyncio.run(main())

# corrected

In [ ]:
import os
import json
import pandas as pd
import numpy as np
import base64
import requests
import asyncio
import aiohttp
import nest_asyncio
import time
from anthropic import Anthropic
import argparse
from PIL import Image
import io
import random
from sklearn.utils import shuffle

nest_asyncio.apply()

# Set the base directory
base_directory = 'DIRECTORY/AgEval-datasets/severity-based-rice-disease/train'
model_type = 'claude'
# FOR API
max_requests_per_half_minute = 5

total_samples_to_check = 10

# Define the list of expected class labels
expected_classes = ['Healthy', 'Mild Bacterial Blight', 'Mild Blast', 'Mild Brownspot', 'Mild Tungro', 'Severe Bacterial Blight', 'Severe Blast', 'Severe Brownspot', 'Severe Tungro']

num_instances_per_class = int(total_samples_to_check/len(expected_classes))
# Create empty lists to store file paths and labels
file_paths = []
labels = []

# Loop through all subdirectories in the base directory
for subdir in os.listdir(base_directory):
    if subdir == ".DS_Store":
        continue
    subdir_path = os.path.join(base_directory, subdir)
    # Loop through all files in the subdirectory
    for filename in os.listdir(subdir_path):
        if filename.lower().endswith(('.jpg', '.jpeg', '.png')):
            # Append the file path and label to the respective lists
            file_paths.append(os.path.join(subdir_path, filename))
            labels.append(subdir)

# Create a dataframe from the lists
data = pd.DataFrame({0: file_paths, 1: labels})

# Get the unique classes
unique_classes = data[1].unique()

# Create a new dataframe with sampled data
sampled_data = pd.DataFrame(columns=[0, 1])

# Sample uniformly from each class
for cls in unique_classes:
    class_data = data[data[1] == cls].sample(n=num_instances_per_class, random_state=42)
    sampled_data = pd.concat([sampled_data, class_data], ignore_index=True)

# Shuffle the sampled data
all_data = shuffle(sampled_data, random_state=42).reset_index(drop=True)

def load_image(image_path: str) -> str:
    """
    Load image from file, convert to JPEG, and encode as base64.
    """
    try:
        with Image.open(image_path) as img:
            # Convert image to RGB mode if it's not
            if img.mode != 'RGB':
                img = img.convert('RGB')
            
            # Save image to a bytes buffer as JPEG
            buffer = io.BytesIO()
            img.save(buffer, format="JPEG", quality=85)  # You can adjust quality as needed
            
            # Encode the JPEG image as base64
            return base64.b64encode(buffer.getvalue()).decode('utf-8')
    except Exception as e:
        print(f"Error processing image {image_path}: {str(e)}")
        return None

vision_prompt = f"""
Given the image, identify the class. Use the following list of possible classes for your prediction It should be one of the : {expected_classes}. Be attentive to subtle details as some classes may appear similar. Provide your answer in the following JSON format:
{{"prediction": "class_name"}}
Replace "class_name" with the appropriate class from the list above based on your analysis of the image.
The labels should be entered exactly as they are in the list above i.e., {expected_classes}.
The response should start with {{ and contain only a JSON object (as specified above) and no other text.
"""

request_times = []

async def get_image_informations_gpt(session, inputs: dict) -> dict:
    """Invoke GPT-4 Vision model with image and prompt."""
    global request_times
    
    while True:
        current_time = time.time()
        
        # Remove old request times outside the current time window
        while request_times and request_times[0] <= current_time - 30:
            request_times.pop(0)
        
        if len(request_times) < max_requests_per_half_minute:
            request_times.append(current_time)
            break
        else:
            # If the rate limit is exceeded, wait for a short duration before checking again
            await asyncio.sleep(1)
    
    api_key = os.getenv("OPENAI_API_KEY")
    model = "gpt-4o-2024-05-13"
    url = "https://api.openai.com/v1/chat/completions"
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {api_key}"
    }
    
    payload = {
        "model": model,
        "messages": [
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": vision_prompt},
                    *inputs['examples'],
                    {
                        "type": "image_url",
                        "image_url": {
                            "url": f"data:image/jpeg;base64,{inputs['image']}",
                            "detail": "high"
                        }
                    }
                ]
            }
        ],
        "max_tokens": 4096
    }
    async with session.post(url, headers=headers, json=payload) as response:
        result = await response.json()
        print(result)
        if "choices" in result and result["choices"]:
            return result["choices"][0]['message']['content']
        else:
            raise Exception("Unexpected API response format")

async def get_image_informations_claude(inputs: dict) -> str:
    """Invoke Claude 3.5 Sonnet model with image and prompt."""
    anthropic_api_key = os.getenv("ANTHROPIC_API_KEY")
    anthropic_client = Anthropic(api_key=anthropic_api_key)
    
    messages = [
        {
            "role": "user",
            "content": [
                {"type": "text", "text": vision_prompt},
                *[
                    {
                        "type": "image",
                        "source": {
                            "type": "base64",
                            "media_type": "image/jpeg",
                            "data": ex['image_url']['url'].split(',')[1] if ex['type'] == 'image_url' else ex['source']['data']
                        }
                    }
                    if ex['type'] in ['image_url', 'image'] else ex
                    for ex in inputs['examples']
                ],
                {
                    "type": "image",
                    "source": {
                        "type": "base64",
                        "media_type": "image/jpeg",
                        "data": inputs['image']
                    }
                }
            ]
        }
    ]
    
    response = anthropic_client.messages.create(
        model="claude-3-sonnet-20240229",
        max_tokens=1000,
        temperature=0.3,
        messages=messages
    )
    return response.content[0].text

async def process_image(session, i, number_of_shots, all_data_results, model_type):
    try:
        image_path = all_data[0][i]
        image_base64 = load_image(image_path)
        if image_base64 is None:
            raise ValueError(f"Failed to load image: {image_path}")
        
        examples = []

        # Get the number of rows in all_data
        num_rows = len(all_data)

        # Select random indices from the dataframe
        random_indices = random.sample([idx for idx in range(num_rows) if idx != i], number_of_shots)

        for j in random_indices:
            example_image_path = all_data[0][j]
            example_image_base64 = load_image(example_image_path)
            if example_image_base64 is not None:
                if model_type == "gpt":
                    examples.append({"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{example_image_base64}", "detail": "high"}})
                elif model_type == "claude":
                    examples.append({
                        "type": "image",
                        "source": {
                            "type": "base64",
                            "media_type": "image/jpeg",
                            "data": example_image_base64
                        }
                    })
                examples.append({"type": "text", "text": f'{{"prediction": "{all_data.at[j, 1]}"}}' })

        if model_type == "gpt":
            prediction = await get_image_informations_gpt(session, {"image": image_base64, "examples": examples})
        elif model_type == "claude":
            prediction = await get_image_informations_claude({"image": image_base64, "examples": examples})
        else:
            raise ValueError(f"Unsupported model type: {model_type}")

        all_data_results.at[i, f"# of Shots {number_of_shots}"] = json.loads(prediction)['prediction']

    except Exception as e:
        print(f"Error processing {all_data[0][i]}: {str(e)}")
        all_data_results.at[i, f"# of Shots {number_of_shots}"] = 'NA'

async def process_images_for_shots(number_of_shots, all_data_results, model_type):
    if model_type == "gpt":
        async with aiohttp.ClientSession() as session:
            tasks = []
            for i in range(len(all_data)):
                task = asyncio.ensure_future(process_image(session, i, number_of_shots, all_data_results, model_type))
                tasks.append(task)
            await asyncio.gather(*tasks)
    elif model_type == "claude":
        for i in range(len(all_data)):
            await process_image(None, i, number_of_shots, all_data_results, model_type)

async def main():
    all_data_results = all_data.copy(deep=True)
    all_data_results.columns = all_data_results.columns.map(str)

    for number_of_shots in [2]:  # [0, 1, 2, 4, 8]:
        await process_images_for_shots(number_of_shots, all_data_results, model_type)
    
    # Create output directory if it doesn't exist
    output_dir = f"results_{model_type}"
    os.makedirs(output_dir, exist_ok=True)
    
    # Save results to the model-specific directory
    output_file = os.path.join(output_dir, f"SBRD_{model_type}.csv")
    all_data_results.to_csv(output_file)
    print(f"Results saved to {output_file}")

if __name__ == "__main__":
    asyncio.run(main())

In [ ]:
# API and Utility Functions
import os
import json
import base64
import asyncio
import aiohttp
import time
from anthropic import Anthropic
from PIL import Image
import io

def load_image(image_path: str) -> str:
    """
    Load image from file, convert to JPEG, and encode as base64.
    """
    try:
        with Image.open(image_path) as img:
            if img.mode != 'RGB':
                img = img.convert('RGB')
            buffer = io.BytesIO()
            img.save(buffer, format="JPEG", quality=85)
            return base64.b64encode(buffer.getvalue()).decode('utf-8')
    except Exception as e:
        print(f"Error processing image {image_path}: {str(e)}")
        return None

class RateLimiter:
    def __init__(self, max_requests, time_window):
        self.max_requests = max_requests
        self.time_window = time_window
        self.request_times = []

    async def wait(self):
        while True:
            current_time = time.time()
            self.request_times = [t for t in self.request_times if t > current_time - self.time_window]
            if len(self.request_times) < self.max_requests:
                self.request_times.append(current_time)
                break
            await asyncio.sleep(0.1)

class GPTAPI:
    def __init__(self, api_key, model="gpt-4o-2024-05-13"):
        self.api_key = api_key
        self.model = model
        self.url = "https://api.openai.com/v1/chat/completions"
        self.headers = {
            "Content-Type": "application/json",
            "Authorization": f"Bearer {self.api_key}"
        }
        self.rate_limiter = RateLimiter(max_requests=5, time_window=30)

    async def get_image_information(self, session, inputs: dict) -> str:
        await self.rate_limiter.wait()
        payload = {
            "model": self.model,
            "messages": [
                {
                    "role": "user",
                    "content": [
                        {"type": "text", "text": inputs['prompt']},
                        *inputs['examples'],
                        {
                            "type": "image_url",
                            "image_url": {
                                "url": f"data:image/jpeg;base64,{inputs['image']}",
                                "detail": "high"
                            }
                        }
                    ]
                }
            ],
            "max_tokens": 4096
        }
        async with session.post(self.url, headers=self.headers, json=payload) as response:
            result = await response.json()
            if "choices" in result and result["choices"]:
                return result["choices"][0]['message']['content']
            else:
                raise Exception("Unexpected API response format")

class ClaudeAPI:
    def __init__(self, api_key, model="claude-3-sonnet-20240229"):
        self.client = Anthropic(api_key=api_key)
        self.model = model

    async def get_image_information(self, inputs: dict) -> str:
        messages = [
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": inputs['prompt']},
                    *[
                        {
                            "type": "image",
                            "source": {
                                "type": "base64",
                                "media_type": "image/jpeg",
                                "data": ex['image_url']['url'].split(',')[1] if ex['type'] == 'image_url' else ex['source']['data']
                            }
                        }
                        if ex['type'] in ['image_url', 'image'] else ex
                        for ex in inputs['examples']
                    ],
                    {
                        "type": "image",
                        "source": {
                            "type": "base64",
                            "media_type": "image/jpeg",
                            "data": inputs['image']
                        }
                    }
                ]
            }
        ]
        response = self.client.messages.create(
            model=self.model,
            max_tokens=1000,
            temperature=0.3,
            messages=messages
        )
        return response.content[0].text

# Main Script
import pandas as pd
import numpy as np
import random
from sklearn.utils import shuffle
import nest_asyncio

nest_asyncio.apply()

# Set the base directory
base_directory = 'DIRECTORY/AgEval-datasets/severity-based-rice-disease/train'
model_type = 'gpt'
total_samples_to_check = 10

# Define the list of expected class labels
expected_classes = ['Healthy', 'Mild Bacterial Blight', 'Mild Blast', 'Mild Brownspot', 'Mild Tungro', 'Severe Bacterial Blight', 'Severe Blast', 'Severe Brownspot', 'Severe Tungro']

def load_and_prepare_data(base_directory, total_samples_to_check):
    num_instances_per_class = int(total_samples_to_check / len(expected_classes))
    file_paths = []
    labels = []

    for subdir in os.listdir(base_directory):
        if subdir == ".DS_Store":
            continue
        subdir_path = os.path.join(base_directory, subdir)
        for filename in os.listdir(subdir_path):
            if filename.lower().endswith(('.jpg', '.jpeg', '.png')):
                file_paths.append(os.path.join(subdir_path, filename))
                labels.append(subdir)

    data = pd.DataFrame({0: file_paths, 1: labels})
    sampled_data = pd.DataFrame(columns=[0, 1])

    for cls in data[1].unique():
        class_data = data[data[1] == cls].sample(n=num_instances_per_class, random_state=42)
        sampled_data = pd.concat([sampled_data, class_data], ignore_index=True)

    return shuffle(sampled_data, random_state=42).reset_index(drop=True)

vision_prompt = f"""
Given the image, identify the class. Use the following list of possible classes for your prediction It should be one of the : {expected_classes}. Be attentive to subtle details as some classes may appear similar. Provide your answer in the following JSON format:
{{"prediction": "class_name"}}
Replace "class_name" with the appropriate class from the list above based on your analysis of the image.
The labels should be entered exactly as they are in the list above i.e., {expected_classes}.
The response should start with {{ and contain only a JSON object (as specified above) and no other text.
"""

async def process_image(api, session, i, number_of_shots, all_data_results, all_data):
    try:
        image_path = all_data[0][i]
        image_base64 = load_image(image_path)
        if image_base64 is None:
            raise ValueError(f"Failed to load image: {image_path}")
        
        examples = []
        num_rows = len(all_data)
        random_indices = random.sample([idx for idx in range(num_rows) if idx != i], number_of_shots)

        for j in random_indices:
            example_image_path = all_data[0][j]
            example_image_base64 = load_image(example_image_path)
            if example_image_base64 is not None:
                if isinstance(api, GPTAPI):
                    examples.append({"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{example_image_base64}", "detail": "high"}})
                elif isinstance(api, ClaudeAPI):
                    examples.append({
                        "type": "image",
                        "source": {
                            "type": "base64",
                            "media_type": "image/jpeg",
                            "data": example_image_base64
                        }
                    })
                examples.append({"type": "text", "text": f'{{"prediction": "{all_data.at[j, 1]}"}}' })

        prediction = await api.get_image_information({"image": image_base64, "examples": examples, "prompt": vision_prompt})
        all_data_results.at[i, f"# of Shots {number_of_shots}"] = json.loads(prediction)['prediction']

    except Exception as e:
        print(f"Error processing {all_data[0][i]}: {str(e)}")
        all_data_results.at[i, f"# of Shots {number_of_shots}"] = 'NA'

async def process_images_for_shots(api, number_of_shots, all_data_results, all_data):
    if isinstance(api, GPTAPI):
        async with aiohttp.ClientSession() as session:
            tasks = []
            for i in range(len(all_data)):
                task = asyncio.ensure_future(process_image(api, session, i, number_of_shots, all_data_results, all_data))
                tasks.append(task)
            await asyncio.gather(*tasks)
    elif isinstance(api, ClaudeAPI):
        for i in range(len(all_data)):
            await process_image(api, None, i, number_of_shots, all_data_results, all_data)

async def main():
    all_data = load_and_prepare_data(base_directory, total_samples_to_check)
    all_data_results = all_data.copy(deep=True)
    all_data_results.columns = all_data_results.columns.map(str)

    if model_type == "gpt":
        api = GPTAPI(api_key=os.getenv("OPENAI_API_KEY"))
    elif model_type == "claude":
        api = ClaudeAPI(api_key=os.getenv("ANTHROPIC_API_KEY"))
    else:
        raise ValueError(f"Unsupported model type: {model_type}")

    for number_of_shots in [0]:  # [0, 1, 2, 4, 8]:
        await process_images_for_shots(api, number_of_shots, all_data_results, all_data)
    
    output_dir = f"results_{model_type}"
    os.makedirs(output_dir, exist_ok=True)
    output_file = os.path.join(output_dir, f"SBRD_{model_type}.csv")
    all_data_results.to_csv(output_file)
    print(f"Results saved to {output_file}")

if __name__ == "__main__":
    asyncio.run(main())

In [ ]:
import os
import json
import base64
import asyncio
import aiohttp
import time
from anthropic import Anthropic
from PIL import Image
import io
import pandas as pd
import numpy as np
import random
from sklearn.utils import shuffle
import nest_asyncio
from tqdm import tqdm

nest_asyncio.apply()

def load_image(image_path: str) -> str:
    """
    Load image from file, convert to JPEG, and encode as base64.
    """
    try:
        with Image.open(image_path) as img:
            if img.mode != 'RGB':
                img = img.convert('RGB')
            buffer = io.BytesIO()
            img.save(buffer, format="JPEG", quality=85)
            return base64.b64encode(buffer.getvalue()).decode('utf-8')
    except Exception as e:
        print(f"Error processing image {image_path}: {str(e)}")
        return None

class RateLimiter:
    def __init__(self, max_requests, time_window):
        self.max_requests = max_requests
        self.time_window = time_window
        self.request_times = []

    async def wait(self):
        while True:
            current_time = time.time()
            self.request_times = [t for t in self.request_times if t > current_time - self.time_window]
            if len(self.request_times) < self.max_requests:
                self.request_times.append(current_time)
                break
            await asyncio.sleep(0.1)

class GPTAPI:
    def __init__(self, api_key, model="gpt-4o-2024-05-13"):
        self.api_key = api_key
        self.model = model
        self.url = "https://api.openai.com/v1/chat/completions"
        self.headers = {
            "Content-Type": "application/json",
            "Authorization": f"Bearer {self.api_key}"
        }
        self.rate_limiter = RateLimiter(max_requests=5, time_window=30)

    async def get_image_information(self, session, inputs: dict) -> str:
        await self.rate_limiter.wait()
        payload = {
            "model": self.model,
            "messages": [
                {
                    "role": "user",
                    "content": [
                        {"type": "text", "text": inputs['prompt']},
                        *inputs['examples'],
                        {
                            "type": "image_url",
                            "image_url": {
                                "url": f"data:image/jpeg;base64,{inputs['image']}",
                                "detail": "high"
                            }
                        }
                    ]
                }
            ],
            "max_tokens": 300
        }
        async with session.post(self.url, headers=self.headers, json=payload) as response:
            result = await response.json()
            if "choices" in result and result["choices"]:
                return result["choices"][0]['message']['content']
            else:
                raise Exception(f"Unexpected API response format: {result}")

class ClaudeAPI:
    def __init__(self, api_key, model="claude-3-sonnet-20240229"):
        self.client = Anthropic(api_key=api_key)
        self.model = model

    async def get_image_information(self, inputs: dict) -> str:
        messages = [
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": inputs['prompt']},
                    *[
                        {
                            "type": "image",
                            "source": {
                                "type": "base64",
                                "media_type": "image/jpeg",
                                "data": ex['image_url']['url'].split(',')[1] if ex['type'] == 'image_url' else ex['source']['data']
                            }
                        }
                        if ex['type'] in ['image_url', 'image'] else ex
                        for ex in inputs['examples']
                    ],
                    {
                        "type": "image",
                        "source": {
                            "type": "base64",
                            "media_type": "image/jpeg",
                            "data": inputs['image']
                        }
                    }
                ]
            }
        ]
        response = self.client.messages.create(
            model=self.model,
            max_tokens=1000,
            temperature=0.3,
            messages=messages
        )
        return response.content[0].text

# Set the base directory
base_directory = 'DIRECTORY/AgEval-datasets/severity-based-rice-disease/train'
model_type = 'gpt'
total_samples_to_check = 10

# Define the list of expected class labels
expected_classes = ['Healthy', 'Mild Bacterial Blight', 'Mild Blast', 'Mild Brownspot', 'Mild Tungro', 'Severe Bacterial Blight', 'Severe Blast', 'Severe Brownspot', 'Severe Tungro']

def load_and_prepare_data(base_directory, total_samples_to_check):
    num_instances_per_class = int(total_samples_to_check / len(expected_classes))
    file_paths = []
    labels = []

    for subdir in os.listdir(base_directory):
        if subdir == ".DS_Store":
            continue
        subdir_path = os.path.join(base_directory, subdir)
        for filename in os.listdir(subdir_path):
            if filename.lower().endswith(('.jpg', '.jpeg', '.png')):
                file_paths.append(os.path.join(subdir_path, filename))
                labels.append(subdir)

    data = pd.DataFrame({0: file_paths, 1: labels})
    sampled_data = pd.DataFrame(columns=[0, 1])

    for cls in data[1].unique():
        class_data = data[data[1] == cls].sample(n=num_instances_per_class, random_state=42)
        sampled_data = pd.concat([sampled_data, class_data], ignore_index=True)

    return shuffle(sampled_data, random_state=42).reset_index(drop=True)

vision_prompt = f"""
Given the image, identify the class. Use the following list of possible classes for your prediction It should be one of the : {expected_classes}. Be attentive to subtle details as some classes may appear similar. Provide your answer in the following JSON format:
{{"prediction": "class_name"}}
Replace "class_name" with the appropriate class from the list above based on your analysis of the image.
The labels should be entered exactly as they are in the list above i.e., {expected_classes}.
The response should start with {{ and contain only a JSON object (as specified above) and no other text.
"""

async def process_image(api, session, i, number_of_shots, all_data_results, all_data):
    try:
        image_path = all_data[0][i]
        image_base64 = load_image(image_path)
        if image_base64 is None:
            raise ValueError(f"Failed to load image: {image_path}")
        
        examples = []
        num_rows = len(all_data)
        random_indices = random.sample([idx for idx in range(num_rows) if idx != i], number_of_shots)

        for j in random_indices:
            example_image_path = all_data[0][j]
            example_image_base64 = load_image(example_image_path)
            if example_image_base64 is not None:
                if isinstance(api, GPTAPI):
                    examples.append({"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{example_image_base64}", "detail": "high"}})
                elif isinstance(api, ClaudeAPI):
                    examples.append({
                        "type": "image",
                        "source": {
                            "type": "base64",
                            "media_type": "image/jpeg",
                            "data": example_image_base64
                        }
                    })
                examples.append({"type": "text", "text": f'{{"prediction": "{all_data.at[j, 1]}"}}' })

        if isinstance(api, GPTAPI):
            prediction = await api.get_image_information(session, {"image": image_base64, "examples": examples, "prompt": vision_prompt})
        else:
            prediction = await api.get_image_information({"image": image_base64, "examples": examples, "prompt": vision_prompt})
        
        try:
            parsed_prediction = json.loads(prediction)['prediction']
        except json.JSONDecodeError:
            print(f"Error parsing JSON for image {image_path}. API response: {prediction}")
            parsed_prediction = 'Error: Invalid JSON'
        except KeyError:
            print(f"Missing 'prediction' key for image {image_path}. API response: {prediction}")
            parsed_prediction = 'Error: Missing prediction'

        all_data_results.at[i, f"# of Shots {number_of_shots}"] = parsed_prediction

    except Exception as e:
        print(f"Error processing {all_data[0][i]}: {str(e)}")
        all_data_results.at[i, f"# of Shots {number_of_shots}"] = f'Error: {str(e)}'

async def process_images_for_shots(api, number_of_shots, all_data_results, all_data):
    if isinstance(api, GPTAPI):
        async with aiohttp.ClientSession() as session:
            tasks = []
            for i in tqdm(range(len(all_data)), desc=f"Processing {number_of_shots}-shot samples"):
                task = asyncio.ensure_future(process_image(api, session, i, number_of_shots, all_data_results, all_data))
                tasks.append(task)
            await asyncio.gather(*tasks)
    elif isinstance(api, ClaudeAPI):
        for i in tqdm(range(len(all_data)), desc=f"Processing {number_of_shots}-shot samples"):
            await process_image(api, None, i, number_of_shots, all_data_results, all_data)

async def main():
    all_data = load_and_prepare_data(base_directory, total_samples_to_check)
    all_data_results = all_data.copy(deep=True)
    all_data_results.columns = all_data_results.columns.map(str)

    if model_type == "gpt":
        api = GPTAPI(api_key=os.getenv("OPENAI_API_KEY"))
    elif model_type == "claude":
        api = ClaudeAPI(api_key=os.getenv("ANTHROPIC_API_KEY"))
    else:
        raise ValueError(f"Unsupported model type: {model_type}")

    for number_of_shots in [0]:  # [0, 1, 2, 4, 8]:
        await process_images_for_shots(api, number_of_shots, all_data_results, all_data)
    
    output_dir = f"results_{model_type}"
    os.makedirs(output_dir, exist_ok=True)
    output_file = os.path.join(output_dir, f"SBRD_{model_type}.csv")
    all_data_results.to_csv(output_file)
    print(f"Results saved to {output_file}")

if __name__ == "__main__":
    asyncio.run(main())

In [ ]:
import os
import json
import base64
import asyncio
import aiohttp
import time
from anthropic import Anthropic
from PIL import Image
import io
import pandas as pd
import numpy as np
import random
from sklearn.utils import shuffle
import nest_asyncio
from tqdm import tqdm

nest_asyncio.apply()

def load_image(image_path: str) -> str:
    """
    Load image from file, convert to JPEG, and encode as base64.
    """
    try:
        with Image.open(image_path) as img:
            if img.mode != 'RGB':
                img = img.convert('RGB')
            buffer = io.BytesIO()
            img.save(buffer, format="JPEG", quality=85)
            return base64.b64encode(buffer.getvalue()).decode('utf-8')
    except Exception as e:
        print(f"Error processing image {image_path}: {str(e)}")
        return None

class RateLimiter:
    def __init__(self, max_requests, time_window):
        self.max_requests = max_requests
        self.time_window = time_window
        self.request_times = []

    async def wait(self):
        while True:
            current_time = time.time()
            self.request_times = [t for t in self.request_times if t > current_time - self.time_window]
            if len(self.request_times) < self.max_requests:
                self.request_times.append(current_time)
                break
            await asyncio.sleep(0.1)

class GPTAPI:
    def __init__(self, api_key, model="gpt-4o-2024-05-13"):
        self.api_key = api_key
        self.model = model
        self.url = "https://api.openai.com/v1/chat/completions"
        self.headers = {
            "Content-Type": "application/json",
            "Authorization": f"Bearer {self.api_key}"
        }
        self.rate_limiter = RateLimiter(max_requests=5, time_window=30)

    async def get_image_information(self, inputs: dict) -> str:
        await self.rate_limiter.wait()
        payload = {
            "model": self.model,
            "messages": [
                {
                    "role": "user",
                    "content": [
                        {"type": "text", "text": inputs['prompt']},
                        *inputs['examples'],
                        {
                            "type": "image_url",
                            "image_url": {
                                "url": f"data:image/jpeg;base64,{inputs['image']}",
                                "detail": "high"
                            }
                        }
                    ]
                }
            ],
            "max_tokens": 300
        }
        async with aiohttp.ClientSession() as session:
            async with session.post(self.url, headers=self.headers, json=payload) as response:
                result = await response.json()
                if "choices" in result and result["choices"]:
                    return result["choices"][0]['message']['content']
                else:
                    raise Exception(f"Unexpected API response format: {result}")

class ClaudeAPI:
    def __init__(self, api_key, model="claude-3-sonnet-20240229"):
        self.client = Anthropic(api_key=api_key)
        self.model = model

    async def get_image_information(self, inputs: dict) -> str:
        messages = [
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": inputs['prompt']},
                    *[
                        {
                            "type": "image",
                            "source": {
                                "type": "base64",
                                "media_type": "image/jpeg",
                                "data": ex['image_url']['url'].split(',')[1] if ex['type'] == 'image_url' else ex['source']['data']
                            }
                        }
                        if ex['type'] in ['image_url', 'image'] else ex
                        for ex in inputs['examples']
                    ],
                    {
                        "type": "image",
                        "source": {
                            "type": "base64",
                            "media_type": "image/jpeg",
                            "data": inputs['image']
                        }
                    }
                ]
            }
        ]
        response = self.client.messages.create(
            model=self.model,
            max_tokens=1000,
            temperature=0.3,
            messages=messages
        )
        return response.content[0].text

# Set the base directory
base_directory = 'DIRECTORY/AgEval-datasets/severity-based-rice-disease/train'
model_type = 'claude'  # Change this to 'claude' to use Claude API
total_samples_to_check = 10

# Define the list of expected class labels
expected_classes = ['Healthy', 'Mild Bacterial Blight', 'Mild Blast', 'Mild Brownspot', 'Mild Tungro', 'Severe Bacterial Blight', 'Severe Blast', 'Severe Brownspot', 'Severe Tungro']

def load_and_prepare_data(base_directory, total_samples_to_check):
    num_instances_per_class = int(total_samples_to_check / len(expected_classes))
    file_paths = []
    labels = []

    for subdir in os.listdir(base_directory):
        if subdir == ".DS_Store":
            continue
        subdir_path = os.path.join(base_directory, subdir)
        for filename in os.listdir(subdir_path):
            if filename.lower().endswith(('.jpg', '.jpeg', '.png')):
                file_paths.append(os.path.join(subdir_path, filename))
                labels.append(subdir)

    data = pd.DataFrame({0: file_paths, 1: labels})
    sampled_data = pd.DataFrame(columns=[0, 1])

    for cls in data[1].unique():
        class_data = data[data[1] == cls].sample(n=num_instances_per_class, random_state=42)
        sampled_data = pd.concat([sampled_data, class_data], ignore_index=True)

    return shuffle(sampled_data, random_state=42).reset_index(drop=True)

vision_prompt = f"""
Given the image, identify the class. Use the following list of possible classes for your prediction It should be one of the : {expected_classes}. Be attentive to subtle details as some classes may appear similar. Provide your answer in the following JSON format:
{{"prediction": "class_name"}}
Replace "class_name" with the appropriate class from the list above based on your analysis of the image.
The labels should be entered exactly as they are in the list above i.e., {expected_classes}.
The response should start with {{ and contain only a JSON object (as specified above) and no other text.
"""

async def process_image(api, i, number_of_shots, all_data_results, all_data):
    try:
        image_path = all_data[0][i]
        image_base64 = load_image(image_path)
        if image_base64 is None:
            raise ValueError(f"Failed to load image: {image_path}")
        
        examples = []
        num_rows = len(all_data)
        random_indices = random.sample([idx for idx in range(num_rows) if idx != i], number_of_shots)

        for j in random_indices:
            example_image_path = all_data[0][j]
            example_image_base64 = load_image(example_image_path)
            if example_image_base64 is not None:
                if isinstance(api, GPTAPI):
                    examples.append({"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{example_image_base64}", "detail": "high"}})
                elif isinstance(api, ClaudeAPI):
                    examples.append({
                        "type": "image",
                        "source": {
                            "type": "base64",
                            "media_type": "image/jpeg",
                            "data": example_image_base64
                        }
                    })
                examples.append({"type": "text", "text": f'{{"prediction": "{all_data.at[j, 1]}"}}' })

        prediction = await api.get_image_information({"image": image_base64, "examples": examples, "prompt": vision_prompt})
        
        try:
            parsed_prediction = json.loads(prediction)['prediction']
        except json.JSONDecodeError:
            print(f"Error parsing JSON for image {image_path}. API response: {prediction}")
            parsed_prediction = 'Error: Invalid JSON'
        except KeyError:
            print(f"Missing 'prediction' key for image {image_path}. API response: {prediction}")
            parsed_prediction = 'Error: Missing prediction'

        all_data_results.at[i, f"# of Shots {number_of_shots}"] = parsed_prediction

    except Exception as e:
        print(f"Error processing {all_data[0][i]}: {str(e)}")
        all_data_results.at[i, f"# of Shots {number_of_shots}"] = f'Error: {str(e)}'

async def process_images_for_shots(api, number_of_shots, all_data_results, all_data):
    tasks = []
    for i in tqdm(range(len(all_data)), desc=f"Processing {number_of_shots}-shot samples"):
        task = asyncio.ensure_future(process_image(api, i, number_of_shots, all_data_results, all_data))
        tasks.append(task)
    await asyncio.gather(*tasks)

async def main():
    all_data = load_and_prepare_data(base_directory, total_samples_to_check)
    all_data_results = all_data.copy(deep=True)
    all_data_results.columns = all_data_results.columns.map(str)

    if model_type == "gpt":
        api = GPTAPI(api_key=os.getenv("OPENAI_API_KEY"))
    elif model_type == "claude":
        api = ClaudeAPI(api_key=os.getenv("ANTHROPIC_API_KEY"))
    else:
        raise ValueError(f"Unsupported model type: {model_type}")

    for number_of_shots in [0]:  # [0, 1, 2, 4, 8]:
        await process_images_for_shots(api, number_of_shots, all_data_results, all_data)
    
    output_dir = f"results_{model_type}"
    os.makedirs(output_dir, exist_ok=True)
    output_file = os.path.join(output_dir, f"SBRD_{model_type}.csv")
    all_data_results.to_csv(output_file)
    print(f"Results saved to {output_file}")

if __name__ == "__main__":
    asyncio.run(main())